# Preprocessing New Mexico Allocation data for WaDE upload.
- Purpose:  To preprocess the New Mexico data into one master file for simple DataFrame creation and extraction.

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/NewMexico/WaterAllocation/RawInputData"
os.chdir(workingDir)

### OSE POD Data

In [3]:
# Input File - OSE POD data
dfinPOD = pd.read_csv('OSE_PODs.zip', compression='zip').replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "nmwr" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('OSE_PODs.zip', compression=dict(method='zip', archive_name='OSE_PODs.csv'), index=False)

print(len(dfinPOD))
dfinPOD.head(1)

C:\Users\rjame\AppData\Local\Temp\ipykernel_16412\3715380858.py:2: DtypeWarning: Columns (2,4,55) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOD = pd.read_csv('OSE_PODs.zip', compression='zip').replace(np.nan, "")


260149


,WaDEUUID,ReasonRemoved,IncompleteField,pod_basin,pod_nbr,pod_suffix,ref,pod_name,tws,rng,sec,qtr_4th,qtr_16th,qtr_64th,blk,zone,x,y,landgrant,legal,county,start_date,plug_date,elevation,depth_well,sched_date,pump_type,discharge,aquifer,sys_date,restrict_,lat_deg,lat_min,lat_sec,lon_deg,lon_min,lon_sec,pod_status,ditch_name,utm_zone,easting,northing,datum,utm_source,xy_source,tract_nbr,map_nbr,surv_map,other_loc,cs_code,wrats_s_id,utm_error,well_tag,pod_file,basin,nbr,suffix,sub_basin,status,use_,total_div,sub_file,sf_header,db_file,own_lname,own_fname,addr1,addr2,city,state,zip,in_state,dump_date,loc_error,wr_count,replaced,OBJECTID_1,OBJECTID,license_nb,finish_dat,pcw_rcv_da,grnd_wtr_s,percent_sh,depth_wate,log_file_d,use_of_wel,pump_seria,subdiv_nam,subdiv_loc,surface_co,estimate_y,casing_siz,utm_accura,xy_accurac,lat_lon_so,lat_lon_ac,pod_rec_nb,cfs_start_,cfs_end_md,cfs_cnv_fa,pod_sub_ba,static_lev,sum_rec_nb,contact_ln,contact_fn,nmwrrs_wrs
0,nmwr0,Incomplete or bad entry for WaterSourceTypeCV,,G,1904,,,,13N,16W,9,1,4,3,,,0.00000,0.00000,,,MK,,,0.00000,25,,,,,2003/11/04 00:00:00+00,0.00000,0,0,0.00000,0,0,0.00000,,,13,178203.00000,3920141.00000,NAD83,G,,,,,,0,0,,,G-01904,G,1904,,G,DCL,STK,1.00000,,,G-01904,BERGER,LAURENCE R. AND DIANA M.,REVOCABLE TRUST,406 BAKER AVE,GALLUP,NM,87301,1,2022/05/31 00:00:00+00,0,1,0,134957,134957,0,1950/01/01 00:00:00+00,,,0,0,,,,,,0,5,0.00000,0,,,M,196552,,,0.00000,G,0,186627,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...


In [4]:
#technique to check datatype of long dataframes.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfinPOD.dtypes)

WaDEUUID            object
ReasonRemoved       object
IncompleteField     object
pod_basin           object
pod_nbr             object
pod_suffix          object
ref                 object
pod_name            object
tws                 object
rng                 object
sec                 object
qtr_4th             object
qtr_16th            object
qtr_64th            object
blk                 object
zone                object
x                  float64
y                  float64
landgrant           object
legal               object
county              object
start_date          object
plug_date           object
elevation          float64
depth_well           int64
sched_date          object
pump_type           object
discharge           object
aquifer             object
sys_date            object
restrict_          float64
lat_deg              int64
lat_min              int64
lat_sec            float64
lon_deg              int64
lon_min              int64
lon_sec            float64
p

In [5]:
print("Project to longitude/ latitude ")

from pyproj import transform, Proj

dfinPOD = dfinPOD.assign(in_Longitude='')
dfinPOD = dfinPOD.assign(in_Latitude='')

utmProj = Proj(init="epsg:26913", proj="utm", zone=13)

lonList = []
latList = []

for ix in range(len(dfinPOD.index)):
    #print(ix)
    x1 = dfinPOD.loc[ix, 'easting']
    y1 = dfinPOD.loc[ix, 'northing']
    try:
        lon, lat = utmProj(float(x1), float(y1), inverse=True)
        lonList.append(lon)
        latList.append(lat)
    except:
        lonList.append('')
        latList.append('')

dfinPOD['in_Longitude'] = lonList
dfinPOD['in_Latitude'] = latList
dfinPOD.head(3)

Project to longitude/ latitude 


C:\Users\rjame\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,WaDEUUID,ReasonRemoved,IncompleteField,pod_basin,pod_nbr,pod_suffix,ref,pod_name,tws,rng,sec,qtr_4th,qtr_16th,qtr_64th,blk,zone,x,y,landgrant,legal,county,start_date,plug_date,elevation,depth_well,sched_date,pump_type,discharge,aquifer,sys_date,restrict_,lat_deg,lat_min,lat_sec,lon_deg,lon_min,lon_sec,pod_status,ditch_name,utm_zone,easting,northing,datum,utm_source,xy_source,tract_nbr,map_nbr,surv_map,other_loc,cs_code,wrats_s_id,utm_error,well_tag,pod_file,basin,nbr,suffix,sub_basin,status,use_,total_div,sub_file,sf_header,db_file,own_lname,own_fname,addr1,addr2,city,state,zip,in_state,dump_date,loc_error,wr_count,replaced,OBJECTID_1,OBJECTID,license_nb,finish_dat,pcw_rcv_da,grnd_wtr_s,percent_sh,depth_wate,log_file_d,use_of_wel,pump_seria,subdiv_nam,subdiv_loc,surface_co,estimate_y,casing_siz,utm_accura,xy_accurac,lat_lon_so,lat_lon_ac,pod_rec_nb,cfs_start_,cfs_end_md,cfs_cnv_fa,pod_sub_ba,static_lev,sum_rec_nb,contact_ln,contact_fn,nmwrrs_wrs,in_Longitude,in_Latitude
0,nmwr0,Incomplete or bad entry for WaterSourceTypeCV,,G,1904,,,,13N,16W,9,1,4,3,,,0.00000,0.00000,,,MK,,,0.00000,25,,,,,2003/11/04 00:00:00+00,0.00000,0,0,0.00000,0,0,0.00000,,,13,178203.00000,3920141.00000,NAD83,G,,,,,,0,0,,,G-01904,G,1904,,G,DCL,STK,1.00000,,,G-01904,BERGER,LAURENCE R. AND DIANA M.,REVOCABLE TRUST,406 BAKER AVE,GALLUP,NM,87301,1,2022/05/31 00:00:00+00,0,1,0,134957,134957,0,1950/01/01 00:00:00+00,,,0,0,,,,,,0,5,0.00000,0,,,M,196552,,,0.00000,G,0,186627,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...,-108.54187,35.37273
1,nmwr1,Incomplete or bad entry for WaterSourceTypeCV,,CL,128,POD1,,,05S,36E,13,3,1,1,,,0.00000,0.00000,,,RO,,,0.00000,180,,,,,2009/12/02 00:00:00+00,0.00000,0,0,0.00000,0,0,0.00000,,,13,673387.00000,3749476.00000,NAD83,G,,,,,,0,0,,,CL-00128-POD1,CL,128,,CL,PMT,IRR,120.00000,,,CL-00128,WATSON,KODDY D,2326 S ROOSEVELT ROAD L,,ROGERS,NM,88132,1,2022/05/31 00:00:00+00,0,1,0,156596,156596,0,1950/12/31 00:00:00+00,,,0,0,,,,,,0,0,16.00000,,,,,223552,,,0.00000,CL,0,220177,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...,-103.12541,33.87138
2,nmwr10,,,RG,68193,,,,12N,03E,26,2,1,,,,0.00000,0.00000,,,SA,1800/01/01 00:00:00+00,,0.00000,80,,,,,1997/08/01 00:00:00+00,0.00000,0,0,0.00000,0,0,0.00000,,,13,355053.00000,3901383.00000,NAD83,G,,44A,15,MRGCD,,0,0,,,RG-68193,RG,68193,,MRG,DCL,IRR,16.52500,,,RG-68193,TARGHETTA,ROBERTA,P. O. BOX 433,,CORRALES,NM,87048,1,2022/05/31 00:00:00+00,0,1,0,74049,74049,0,1800/01/01 00:00:00+00,,S,100,8,,IRRIGATION,,,,0,0,2.00000,,,,,105170,,,0.00000,MRG,0,103768,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...,-106.59313,35.24506


In [6]:
# For creating Water source type

dfinPOD.assign(GroundWaterSourceTypeCV='')
dfinPOD.assign(SurfaceWaterSourceTypeCV='')

dfinPOD["GroundWaterSourceTypeCV"] = dfinPOD.apply(lambda row: 'Groundwater' 
                                             if str(row["grnd_wtr_s"]).strip() != '' 
                                             else '', axis=1)
dfinPOD["SurfaceWaterSourceTypeCV"] =dfinPOD.apply(lambda row: 'Surface Water' 
                                              if int(row["surface_co"]) > 0 
                                              else '', axis=1)



dfinPOD.assign(WaterSourceTypeCV='')

waterSourceList = []
for index, row in dfinPOD.iterrows():
    #print(index)
    groundWatetrStr = row['GroundWaterSourceTypeCV']
    surfaceWatetrStr = row['SurfaceWaterSourceTypeCV']
    if (groundWatetrStr != '') & (surfaceWatetrStr != ''):
        waterSourceList.append("Surface and Groundwater")
    elif groundWatetrStr != '':
        waterSourceList.append(groundWatetrStr)
    elif surfaceWatetrStr != '':
        waterSourceList.append(surfaceWatetrStr)
    else:
        waterSourceList.append("")

dfinPOD['in_WaterSourceTypeCV'] = waterSourceList
dfinPOD['in_WaterSourceTypeCV'].unique()

array(['', 'Groundwater', 'Surface Water', 'Surface and Groundwater'],
      dtype=object)

In [7]:
#WaterSourceName

waterSourceDict = {
"1":"ABBOT CREEK",
"2":"ABBOTT LAKE",
"3":"ABIQUIU CREEK",
"4":"ABIQUIU RESERVOIR",
"5":"ABO ARROYO",
"6":"ADAMS CANYON",
"7":"AGUA CALIENTE",
"8":"AGUA CHIQUITA CREEK",
"9":"AGUA FRIA CREEK",
"10":"AGUA FRIA CREEK",
"11":"AGUA FRIA CREEK /N",
"12":"AGUA LIMPIA /N",
"13":"AGUA MEDIA",
"14":"AGUA NEGRA RIVER /N",
"15":"AGUA OLYMPIA",
"16":"AGUA SARCA",
"17":"AGUAJE ARROYO /NM",
"18":"AGUILA CANYON",
"19":"AHOGADERO CREEK",
"20":"ALAMAGORDO RESERVOIR",
"21":"ALAMITO CREEK",
"22":"ALAMITOS CANYON",
"23":"ALAMITOS CREEK",
"24":"ALAMO ARROYO",
"25":"ALAMO CANYON",
"26":"ALAMO CANYON",
"27":"ALAMO CANYON",
"28":"ALAMO CREEK",
"29":"ALAMO CREEK",
"30":"ALAMOCITA ARROYO",
"31":"ALAMOCITA CREEK",
"32":"ALAMOCITA CREEK",
"33":"ALAMOCITA CREEK",
"34":"ALAMOCITO CREEK",
"35":"ALAMOGORDO RESERVOIR",
"36":"ALAMOS CREEK",
"37":"ALAMOS CREEK",
"38":"ALAMOSA",
"39":"ALAMOSA CANYON",
"40":"ALAMOSA CREEK",
"41":"ALAMOSA CREEK",
"42":"ALAMOSA CREEK",
"43":"ALAMOSITA CREEK",
"44":"ALAMOSITA CREEK",
"45":"ALEMAN DRAW",
"46":"ALEXANDER CANYON",
"47":"ALIES SEEP CANYON",
"48":"ALKALI DRAW",
"49":"ALLIE CANYON",
"50":"ALLISON DRAW /NM",
"51":"AMARGO CREEK",
"52":"AMBROSIA LAKE",
"53":"AMERICAN CANYON",
"54":"AMERICAN CREEK",
"55":"AMOLE ARROYO",
"56":"ANAN CANYON",
"57":"ANCHA GULCH",
"58":"ANCHO VALLEY",
"59":"ANGOSTURA CREEK",
"60":"ANIMAS BASIN",
"61":"ANIMAS CREEK",
"62":"ANIMAS RIVER",
"63":"ANTHONY ARROYO",
"64":"APACHE ARROYO",
"65":"APACHE CANYON",
"66":"APACHE CANYON",
"67":"APACHE CREEK",
"68":"APACHE CREEK",
"69":"APACHE CREEK",
"70":"APACHE HILL ARROYO /NM",
"71":"APODACA ARROYO",
"72":"ARAGON CREEK",
"73":"ARCHULETA CREEK",
"74":"ARENAL GRAVEL PIT",
"75":"ARMIJO DRAW",
"76":"ARROW CANYON",
"77":"ARROYO AGUA SARCA",
"78":"ARROYO AGUAJE DE LA PETACA",
"79":"ARROYO AJUELOS",
"80":"ARROYO ALAMITO",
"81":"ARROYO ALCALDE",
"82":"ARROYO ANGOSTURA",
"83":"ARROYO BARBARA",
"84":"ARROYO BLANCO",
"85":"ARROYO CALABASAS",
"86":"ARROYO CANADA ANCHA",
"87":"ARROYO CHICO",
"88":"ARROYO COLORADO",
"89":"ARROYO COMANCHE",
"90":"ARROYO CUARAI /N",
"91":"ARROYO CUERVO",
"92":"ARROYO DE ANIL",
"93":"ARROYO DE FRIJOLES",
"94":"ARROYO DE LA BORREGOS",
"95":"ARROYO DE LA CEJITA",
"96":"ARROYO DE LA MORA",
"97":"ARROYO DE LA MORADA",
"98":"ARROYO DE LA PRESILLA",
"99":"ARROYO DE LAS CRUCES",
"100":"ARROYO DE LAS PALOMAS",
"101":"ARROYO DE LOS ANGELES",
"102":"ARROYO DE LOS CHAMISOS",
"103":"ARROYO DE LOS LOPEZ",
"104":"ARROYO DE LOS TANQUES",
"105":"ARROYO DE MANZANO",
"106":"ARROYO DE TAJIQUE",
"107":"ARROYO DEL ALAMO",
"108":"ARROYO DEL ALAMO",
"109":"ARROYO DEL COYOTE",
"110":"ARROYO DEL CUERVO",
"111":"ARROYO DEL EMBUDO",
"112":"ARROYO DEL GUIQUE",
"113":"ARROYO DEL MACHO",
"114":"ARROYO DEL OJO DEL ORNO",
"115":"ARROYO DEL YESO",
"116":"ARROYO DOMINGO BACA",
"117":"ARROYO ENCINOS",
"118":"ARROYO ESTACA",
"119":"ARROYO GONZALES /NM",
"120":"ARROYO HONDO",
"121":"ARROYO JALAROSA",
"122":"ARROYO JARIDO",
"123":"ARROYO JAROSA /N",
"124":"ARROYO MARTINEZ",
"125":"ARROYO MONTE LARGO",
"126":"ARROYO PECOS",
"127":"ARROYO PEDRO PADILLA",
"128":"ARROYO PIRA",
"129":"ARROYO PUEBLITO",
"130":"ARROYO PUNCHE",
"131":"ARROYO SALEDO",
"132":"ARROYO SAN ANTONIO",
"133":"ARROYO SAN JOSE /N",
"134":"ARROYO SAN JUAN DE DIOS",
"135":"ARROYO SAN RAFAEL",
"136":"ARROYO SECCION",
"137":"ARROYO SECO",
"138":"ARROYO SECO",
"139":"ARROYO SECO",
"140":"ARROYO SECO",
"141":"ARROYO SECO",
"142":"ARROYO SERRANO",
"143":"ARROYO TAJIQUE",
"144":"ARROYO TONQUE",
"145":"ARROYO TRUJILLO /N",
"146":"ARROYO UNA DE GATO",
"147":"ARROYO VAQUEROS",
"148":"ASH CANYON /N",
"149":"ASH CREEK",
"150":"ASH SPRING CANYON",
"151":"ATARQUE CREEK",
"152":"BALDY MOUNTAIN CANYON",
"153":"BALES CANYON",
"154":"BANDERITAS CREEK",
"155":"BAR B DRAW",
"156":"BARCLAY DRAW",
"157":"BARCLAY DRAW",
"158":"BARELA CANYON",
"159":"BARILLAS CREEK",
"160":"BARKER ARROYO",
"161":"BARRANCA CREEK",
"162":"BARRANCONES CREEK",
"163":"BARTON ARROYO",
"164":"BAYLOR CANYON",
"165":"BEAR CANYON",
"166":"BEAR CANYON",
"167":"BEAR CANYON",
"168":"BEAR CANYON",
"169":"BEAR CANYON",
"170":"BEAR CANYON /N",
"171":"BEAR CANYON /N",
"172":"BEAR CANYON /N",
"173":"BEAR CANYON /N",
"174":"BEAR CANYON /N",
"175":"BEAR CREEK",
"176":"BEAR CREEK",
"177":"BEAR CREEK",
"178":"BEAR GRASS DRAW",
"179":"BEAR SPRINGS CANYON",
"180":"BEAVER CANYON",
"181":"BEAVER CREEK",
"182":"BEAVER CREEK",
"183":"BEEN DRAW",
"184":"BELL CANYON",
"185":"BENADO CANYON",
"186":"BENNETT CREEK",
"187":"BENSON CANYON",
"188":"BERCHAM DRAW",
"189":"BERRENDA CREEK",
"190":"BERRENDO CREEK",
"191":"BIG CANYON",
"192":"BIG CANYON",
"193":"BIG CHERRY CANYON",
"194":"BIG CREEK",
"195":"BIG DOG CANYON",
"196":"BIG DRAW",
"197":"BIG DRY CREEK",
"198":"BIG NIGGER GULCH /NM",
"199":"BIG OX YOKE CANYON",
"200":"BIG PAT CANYON",
"201":"BIG PIGEON CANYON",
"202":"BIG RINCON",
"203":"BIGNELL ARROYO",
"204":"BIRCHER CANYON",
"205":"BISBEE DRAW",
"206":"BISHOPS CAP ARROYO",
"207":"BITTER CREEK",
"208":"BITTER CREEK /N",
"209":"BLACK BILL CANYON",
"210":"BLACK CANYON",
"211":"BLACK CANYON",
"212":"BLACK CANYON",
"213":"BLACK CREEK (IN ARIZONA)",
"214":"BLACK MESA ARROYO",
"215":"BLACK MOUNTAIN DRAW",
"216":"BLACK RIVER",
"217":"BLACKSMITH CANYON",
"218":"BLANCA CREEK",
"219":"BLANCO CANYON",
"220":"BLANCO CANYON",
"221":"BLEA STREAM /N",
"222":"BLUE CANYON",
"223":"BLUE CREEK",
"224":"BLUE RIVER",
"225":"BLUE ROCK CANYON",
"226":"BLUE WATER CREEK",
"227":"BLUEWATER CANYON",
"228":"BLUEWATER CREEK",
"229":"BLUEWATER CREEK",
"230":"BLUEWATER CREEK",
"231":"BLUEWATER LAKE",
"232":"BLUFF CREEK",
"233":"BOB CROSBY DRAW",
"234":"BOBCAT CREEK",
"235":"BOHANNON ARROYO",
"236":"BOLANDER CANYON",
"237":"BONANZA CREEK",
"238":"BONITA CANYON",
"239":"BONITA CREEK",
"240":"BONITO CREEK",
"241":"BONITO LAKE",
"242":"BONTZ ARROYO",
"243":"BORICA DRAW",
"244":"BORREGO CANYON",
"245":"BOX CANYON",
"246":"BOX CANYON",
"247":"BOX CANYON",
"248":"BOX CANYON",
"249":"BOX S CANYON",
"250":"BOYER GULCH",
"251":"BRACKETT ARROYO /NM",
"252":"BRAGG CANYON",
"253":"BRANTLEY RESERVOIR",
"254":"BRAZOS RIVER",
"255":"BREAD SPRINGS WASH",
"256":"BRIDGE CANYON",
"257":"BRIGGS CANYON",
"258":"BROAD CANYON",
"259":"BROADHURST ARROYO",
"260":"BRUSHY CANYON",
"261":"BRUSHY CREEK",
"262":"BRUSHY DRAW",
"263":"BUCK CANYON",
"264":"BUCK SPRINGS",
"265":"BUCKHORN CREEK",
"266":"BUEYEROS CREEK",
"267":"BUFFALO DRAW",
"268":"BUFFALO LAKE",
"269":"BUG SCUFFLE CANYON",
"270":"BULL CANYON",
"271":"BULL CANYON CREEK",
"272":"BULL CREEK",
"273":"BULL CREEK",
"274":"BULLARD PEAK CANYON",
"275":"BURNED CANYON",
"276":"BURRO CIENAGA",
"277":"BUSHNELL CREEK",
"278":"BUZZARD CANYON",
"279":"CABALLERO CANYON",
"280":"CABALLO RESERVOIR",
"281":"CABIN CANYON",
"282":"CABRESTO CANYON",
"283":"CABRESTO CREEK",
"284":"CALABACILLAS ARROYO",
"285":"CALAVERAS CANYON",
"286":"CAMERON CREEK",
"287":"CAMP TWO CANYON",
"288":"CAMPUS ARROYO",
"289":"CANADA ALAMOS",
"290":"CANADA ANCHA",
"291":"CANADA BONITA",
"292":"CANADA COLORADO",
"293":"CANADA DE LA CUEVA",
"294":"CANADA DE LA PRESA",
"295":"CANADA DE LAS FUERTES",
"296":"CANADA DEL BANO",
"297":"CANADA DEL BORREGO",
"298":"CANADA DEL PORTRERO",
"299":"CANADA LARGA",
"300":"CANADA PINABETE",
"301":"CANADA RAMONES",
"302":"CANADA SANTIAGO",
"303":"CANADA TIO GRANDE",
"304":"CANADIAN AREA 01",
"305":"CANADIAN AREA 03",
"306":"CANADIAN AREA 06",
"307":"CANADIAN AREA 09",
"308":"CANARIO CANYON",
"309":"CANE SPRING CANYON",
"310":"CANJILON CREEK",
"311":"CANON ANCHO /N",
"312":"CANON BLANCO",
"313":"CANON BONITO /N",
"314":"CANON CEBOLLITA",
"315":"CANON COLORADO",
"316":"CANON CORRALES",
"317":"CANON DE BARTOLO",
"318":"CANON DE CALIFIA",
"319":"CANON DE CHILLILI",
"320":"CANON DE DOMINGA BACA",
"321":"CANON DE GALLEGOS",
"322":"CANON DE HUGHES",
"323":"CANON DE LA CANADA",
"324":"CANON DE LA MIGA",
"325":"CANON DE LA MULA",
"326":"CANON DE LAS PALAS",
"327":"CANON DE LOS CORDOVAS",
"328":"CANON DE LOS PINO REALES",
"329":"CANON DE MARQUEZ",
"330":"CANON DE PEDIO PADILLA",
"331":"CANON DE SALAS",
"332":"CANON DE SALAS /N",
"333":"CANON DE TAJIQUE",
"334":"CANON DE TANQUE HONDO",
"335":"CANON DE TERRERO",
"336":"CANON DE TORREON",
"337":"CANON DEL AGUA",
"338":"CANON DEL BUEY",
"339":"CANON DEL DADO",
"340":"CANON DEL NORTE",
"341":"CANON DEL OJO DEL INDIO",
"342":"CANON DEL TRIGO",
"343":"CANON LARGO",
"344":"CANON MADERA",
"345":"CANON MESTENO",
"346":"CANON MONTE DE ABAJO",
"347":"CANON MONTE LARGO",
"348":"CANON NUEVO",
"349":"CANON OBSCURO",
"350":"CANON OBSCURO",
"351":"CANON SALADO",
"352":"CANON SANTA ROSA",
"353":"CANON SANTO DOMINGO",
"354":"CANON SAPATA",
"355":"CANON SECO",
"356":"CANON SEGURO",
"357":"CANON TAPIA",
"358":"CANONCITO CREEK",
"359":"CANONCITO DE NUANES /N",
"360":"CANONES CREEK /N",
"361":"CANONES CREEK /N",
"362":"CANORITA DE LAS BACAS",
"363":"CANOVAS CREEK",
"364":"CANYON CREEK /NM",
"365":"CAPULIN CANYON",
"366":"CAPULIN CANYON",
"367":"CAPULIN CREEK /N",
"368":"CARACITA CREEK",
"369":"CARLISLE CANYON",
"370":"CARNUDOS DRAW",
"371":"CARRACAS CANYON",
"372":"CARRISA CANYON",
"373":"CARRIZO ARROYO /N",
"374":"CARRIZO CREEK",
"375":"CARRIZO CREEK",
"376":"CARRIZO CREEK",
"377":"CARRIZO CREEK",
"378":"CARRIZO CREEK /N",
"379":"CARRIZO CREEK /N",
"380":"CARRIZO WASH",
"381":"CARRIZOZO CREEK",
"382":"CARRIZOZO CREEK /N",
"383":"CARROS CREEK",
"384":"CARROS CREEK",
"385":"CASAMERO DRAW",
"386":"CASS DRAW",
"387":"CEBOLLA CREEK",
"388":"CEBOLLITA CREEK",
"389":"CEDAR CREEK",
"390":"CEDAR CREEK",
"391":"CEDAR GROVE DRAW",
"392":"CEDRITO ARROYO",
"393":"CELSO ARROYO /N",
"394":"CEMENT CANYON /NM",
"395":"CENEGA DEL MACHO",
"396":"CENTERFIRE CREEK",
"397":"CEREZA CANYON",
"398":"CERROSOSO CREEK",
"399":"CHACO RIVER",
"400":"CHACO WASH",
"401":"CHALK BLUFF DRAW",
"402":"CHAMISAL CREEK",
"403":"CHANEY ARROYO",
"404":"CHAPO DRAW /NM",
"405":"CHARETTE LAKE",
"406":"CHARLEY WHITE DRAW",
"407":"CHATFIELD CANYON",
"408":"CHAVEZ CANYON",
"409":"CHAVEZ CANYON",
"410":"CHAVEZ CREEK",
"411":"CHEROKEE BILL CANYON",
"412":"CHEROKEE CANYON",
"413":"CHERRY CREEK",
"414":"CHERRY CREEK",
"415":"CHESS DRAW",
"416":"CHICO CREEK",
"417":"CHICO RICO CREEK /N",
"418":"CHICORICA CREEK",
"419":"CHICOSA LAKE",
"420":"CHIHUAHUENOS CREEK",
"421":"CHINA DRAW",
"422":"CHINA DRAW /N",
"423":"CHINA DRAW /N",
"424":"CHINA POND DRAW",
"425":"CHOSIE CANYON",
"426":"CHUPADERA ARROYO",
"427":"CHUPADERA ARROYO",
"428":"CHURCH CANYON",
"429":"CIBOLO CANYON",
"430":"CIENEGA AMARILLA",
"431":"CIENEGA CANON /NM",
"432":"CIENEGA CANYON",
"433":"CIENEGA CANYON",
"434":"CIENEGA CREEK",
"435":"CIENEGA DRAW",
"436":"CIENEGUILLA CREEK",
"437":"CIENEGUILLA CREEK",
"438":"CIENEGUILLA DEL BURRO ARROYO",
"439":"CIMARRON CREEK",
"440":"CIMARRON RIVER",
"441":"CIMARRONCITO CREEK",
"442":"CIRUELA CREEK /N",
"443":"CLANTON DRAW",
"444":"CLARK CANYON",
"445":"CLARK DRAW",
"446":"CLARKE ARROYO /N",
"447":"CLEAR CREEK",
"448":"CLEAR LAKE",
"449":"CLIMAX CANYON",
"450":"CLOVERDALE CREEK",
"451":"COAL CANYON",
"452":"COALBANK CANYON",
"453":"COCHITI CANYON /N",
"454":"COCKLEBURR DRAW",
"455":"COFFELT DRAW",
"456":"COLE CANYON /N",
"457":"COLLEGE ARROYO",
"458":"COLLINS DRAW",
"459":"COLUMBINE CREEK",
"460":"COMANCHE CREEK",
"461":"COMANCHE CREEK",
"462":"COMANCHEROS CREEK",
"463":"COMMISSARY CREEK",
"464":"CONCHAS CANAL",
"465":"CONCHAS CANYON",
"466":"CONCHAS LAKE",
"467":"CONCHAS LAKE",
"468":"CONCHAS RIVER",
"469":"COPELAND CANYON",
"470":"COPPER CREEK",
"471":"CORAZON CREEK",
"472":"CORDELL CANYON",
"473":"CORDUROY CANYON",
"474":"CORNUCOPIA DRAW",
"475":"CORNUDAS DRAW",
"476":"CORRAL CANYON",
"477":"CORRAL CANYON",
"478":"CORRALITOS CREEK",
"479":"CORRUMPA CREEK",
"480":"COSTILLA CREEK",
"481":"COTTON CANYON",
"482":"COTTONWOOD ARROYO /N",
"483":"COTTONWOOD CANYON",
"484":"COTTONWOOD CANYON",
"485":"COTTONWOOD CANYON",
"486":"COTTONWOOD CANYON",
"487":"COTTONWOOD CANYON",
"488":"COTTONWOOD CANYON /N",
"489":"COTTONWOOD CREEK",
"490":"COTTONWOOD CREEK",
"491":"COTTONWOOD CREEK",
"492":"COTTONWOOD CREEK /N",
"493":"COTTONWOOD DRAW",
"494":"COTTONWOOD WASH",
"495":"COUNTY LINE ARROYO",
"496":"COW CREEK",
"497":"COW CREEK",
"498":"COW SPRINGS DRAW",
"499":"COX CANYON",
"500":"COX DRAW",
"501":"COYOTE CANYON",
"502":"COYOTE CANYON",
"503":"COYOTE CANYON",
"504":"COYOTE CREEK",
"505":"COYOTE CREEK",
"506":"COYOTE CREEK",
"507":"CRAWFORD HOLLOW",
"508":"CROOKED CANYON",
"509":"CROOKED CREEK",
"510":"CROOKED CREEK",
"511":"CROW CANYON",
"512":"CROW CANYON",
"513":"CROW CANYON",
"514":"CROW CREEK",
"515":"CROW FLATS /N",
"516":"CUCHILLO NEGRO CREEK",
"517":"CUERVO CANYON",
"518":"CUERVO CREEK",
"519":"CUEVO CANYON /N",
"520":"CUNNINGHAM CREEK",
"521":"CURTIS CANYON",
"522":"CURTIS CREEK",
"523":"D BAR O CANYON",
"524":"DALTON CANYON",
"525":"DAM CANYON",
"526":"DARK CANYON /N",
"527":"DARK CANYON /N",
"528":"DARLING CREEK",
"529":"DAVIS CANYON",
"530":"DAVIS CREEK",
"531":"DEAD COW CANYON",
"532":"DEADMAN CANYON",
"533":"DECKER DRAW",
"534":"DEEP CANYON",
"535":"DEEP CREEK",
"536":"DEEP LAKE",
"537":"DEER CREEK",
"538":"DEER CREEK",
"539":"DEER CREEK",
"540":"DEER CREEK",
"541":"DEL MUERTO CREEK",
"542":"DELAWARE RIVER",
"543":"DERRICK DRAW",
"544":"DEVILS CREEK",
"545":"DEVILS DEN CANYON",
"546":"DEVILS NORTH FORK",
"547":"DIAMOND CREEK",
"548":"DICKEY CANYON",
"549":"DILLMAN CREEK",
"550":"DILLON CANYON",
"551":"DIMMIT LAKE",
"552":"DIX CANYON",
"553":"DOCTOR ARROYO",
"554":"DOG CANYON",
"555":"DOG CANYON",
"556":"DOG CANYON DRAW",
"557":"DOLLINS CANYON",
"558":"DOMINGA CANYON",
"559":"DOMINGA CANYON",
"560":"DOMINGUEZ CANYON",
"561":"DONA ANA ARROYO",
"562":"DONAHUE CANYON",
"563":"DOSS ARROYO /N",
"564":"DOUBLE ADOBE CREEK",
"565":"DOUBLE CANYON DRAW",
"566":"DOYLE CANYON",
"567":"DRY ARROYO",
"568":"DRY BURNT CANYON",
"569":"DRY CANYON",
"570":"DRY CIMARRON RIVER",
"571":"DUCK CREEK",
"572":"DUD CANYON",
"573":"DULCE DRAW",
"574":"DULCE DRAW",
"575":"DURAN CANYON",
"576":"DURAN CANYON",
"577":"DURAN MESA",
"578":"DUWESS CANYON",
"579":"EAGLE CREEK /N",
"580":"EAGLE CREEK /N",
"581":"EAGLE DRAW /N",
"582":"EAGLE NEST LAKE",
"583":"EAST CANYON",
"584":"EAST CEDAR CREEK",
"585":"EAST FORK BRUSHY CREEK",
"586":"EAST FORK CHICORICA CREEK",
"587":"EAST FORK FIVEMILE DRAW",
"588":"EAST FORK GILA RIVER",
"589":"EAST FORK JEMEZ RIVER",
"590":"EAST FORK MASON DRAW /N",
"591":"EAST FORK MIMBRES RIVER",
"592":"EAST FORK RED RIVER",
"593":"EAST FORK WINDHAM CANYON",
"594":"EAST SAN JUAN /NM",
"595":"EIGHTMILE DRAW",
"596":"EL CANON DEL PALO AMARILLO",
"597":"EL RANCHO GRANDE DAM /N",
"598":"EL RITO",
"599":"EL RITO CANYON",
"600":"EL RITO CREEK /N",
"601":"EL VADO RESERVOIR",
"602":"ELEPHANT BUTTE RESERVOIR",
"603":"ELK CANYON",
"604":"ELLIOT CANYON",
"605":"ELLIS CANYON",
"606":"EMBUDITO CANYON",
"607":"EMBUDO ARROYO",
"608":"EMBUDO CANYON",
"609":"EMBUDO CREEK",
"610":"EMERSON CANYON",
"611":"ENCINAL CREEK",
"612":"ESCAVADA WASH",
"613":"ESCAVADA WASH",
"614":"ESCONDIDO CANYON",
"615":"ESCRITO CANYON",
"616":"ESQUIBEL CANYON",
"617":"ESTANCIA BASIN",
"618":"ESTEROS CREEK",
"619":"FARMINGTON GLADE",
"620":"FAULKNER CANYON",
"621":"FENCED UP HORSE VALLEY",
"622":"FERNANDEZ DRAW /NM",
"623":"FILLMORE ARROYO",
"624":"FINCH ARROYO",
"625":"FINGER RING DRAW",
"626":"FIR CANYON",
"627":"FIRST VALLEY",
"628":"FISHER CREEK",
"629":"FITZPATRICK CIENEGA",
"630":"FIVEMILE CREEK",
"631":"FIVEMILE DRAW",
"632":"FLATHEAD CANYON",
"633":"FLEMING CANYON",
"634":"FLEMING DRAW",
"635":"FLORIDA RIVER",
"636":"FORBES CANYON",
"637":"FORT CUMMINGS DRAW",
"638":"FORT WEST DITCH",
"639":"FOSTER CANYON",
"640":"FOSTER DRAW",
"641":"FOSTER DRAW",
"642":"FOURMILE CANYON",
"643":"FOURMILE DRAW",
"644":"FRANCES CREEK",
"645":"FRESNAL CANYON",
"646":"FRIO DRAW",
"647":"FROELICH CREEK /N",
"648":"FROLIC CREEK",
"649":"FROST CREEK",
"650":"FRYING PAN CANYON",
"651":"FULLERTON SPRING CANYON",
"652":"FULLERTON SPRING CANYON",
"653":"GALISTEO CREEK",
"654":"GALLEGOS CANYON",
"655":"GALLEGOS CREEK",
"656":"GALLEGOS CREEK /N",
"657":"GALLINA CREEK",
"658":"GALLINAS CANYON",
"659":"GALLINAS CREEK",
"660":"GALLINAS CREEK /N",
"661":"GALLINAS RIVER /N",
"662":"GALLO ARROYO",
"663":"GALLO CANYON",
"664":"GARAPATA CREEK",
"665":"GARCIA CANYON",
"666":"GARCIA CANYON",
"667":"GARCIA CANYON",
"668":"GARCIA CREEK",
"669":"GARCIA FALLS CANYON",
"670":"GARFIELD CANYON",
"671":"GARITA CREEK",
"672":"GATLIN CANYON",
"673":"GAVILAN ARROYO",
"674":"GAVILAN CANON",
"675":"GAVILAN CANYON",
"676":"GEORGE CANYON",
"677":"GERONIMO DRAW",
"678":"GILA RIVER AREA 1",
"679":"GILA RIVER AREA 2",
"680":"GILLESPIE CREEK",
"681":"GILLESPIE DRAW",
"682":"GLORIETA CREEK",
"683":"GOAT CANYON",
"684":"GOAT CANYON",
"685":"GOBERNADOR CANYON",
"686":"GODFREY ARROYO /NM",
"687":"GOLD GULCH",
"688":"GOLD HILL CANYON",
"689":"GONLANS CREEK /NM",
"690":"GONZALES CANYON",
"691":"GONZALES CANYON",
"692":"GOOSE CREEK",
"693":"GOOSE LAKE",
"694":"GOVINA CANYON",
"695":"GRANEY CREEK",
"696":"GRAPEVINE CANYON",
"697":"GRAPEVINE CREEK",
"698":"GRAPEVINE DRAW",
"699":"GRAVEYARD CANYON",
"700":"GREASY CANYON",
"701":"GREEN CANYON",
"702":"GREENFIELD ARROYO /N",
"703":"GREENHORN ARROYO",
"704":"GREENWOOD CANYON",
"705":"GUADALUPE ARROYO",
"706":"GUADALUPE CANYON",
"707":"GUADALUPE CANYON",
"708":"GUADALUPE DRAW",
"709":"GUADALUPITA CREEK /N",
"710":"GUANA CREEK",
"711":"GUERRERO CANYON /N",
"712":"HACHITA VALLEY",
"713":"HACHITA VALLEY",
"714":"HACKBERRY DRAW",
"715":"HACKBERRY DRAW",
"716":"HACKBERRY GULCH /NM",
"717":"HACKETT CANYON",
"718":"HADLEY CANYON",
"719":"HADLEY DRAW",
"720":"HAGERMAN CANAL",
"721":"HAHN ARROYO",
"722":"HAMMETT ARROYO",
"723":"HAMPTON DRAW",
"724":"HANOVER CREEK",
"725":"HARDCASTLE CANYON",
"726":"HARDEN CIENEGA",
"727":"HARRIS CANYON",
"728":"HARRIS CREEK",
"729":"HART CANYON",
"730":"HARVEY DRAW",
"731":"HASPAROS CANYON /N",
"732":"HASPEROS CANYON",
"733":"HAY CANYON",
"734":"HAY DRAW",
"735":"HAYNES CANYON",
"736":"HAYNES CANYON",
"737":"HAZZARDS CANYON /NM",
"738":"HELL CANYON",
"739":"HERNANDEZ DRAW",
"740":"HEWITT CREEK /NM",
"741":"HILL CANYON",
"742":"HINCHLEY CANYON",
"743":"HITTSON CREEK",
"744":"HOGADERO DRAW",
"745":"HOLKEO CREEK",
"746":"HOLLENBACK CREEK",
"747":"HOLY GHOST CREEK",
"748":"HOMBRILLO",
"749":"HOMESTEAD TANK",
"750":"HONDO CANYON",
"751":"HONDO CANYON",
"752":"HOOPER CANYON",
"753":"HOP CANYON",
"754":"HORSE CAMP CANYON",
"755":"HORSE LAKE CANYON",
"756":"HORSE LAKE CREEK",
"757":"HORSEHEAD CANYON",
"758":"HORSESHOE LAKE",
"759":"HOT SPRINGS CANYON",
"760":"HOUGHTON CANYON",
"761":"HOUSE CANYON",
"762":"HOYT CREEK",
"763":"HUBBLE LAKE",
"764":"HUGGINS CREEK",
"765":"HUGGINS DRAW",
"766":"HUGHS CANYON",
"767":"HUMPHREY CANYON",
"768":"HUNTER CREEK",
"769":"HUNTER WASH",
"770":"HUTCH ARROYO",
"771":"HYATT CANYON",
"772":"INDIAN CREEK",
"773":"INDIAN CREEK",
"774":"INDIAN CREEK",
"775":"INDIAN CREEK",
"776":"INDIAN CREEK",
"777":"INDIAN DRAW",
"778":"INDIAN RIDGE CANYON /NM",
"779":"IRON CREEK",
"780":"IRON CREEK",
"781":"IRWIN CREEK /N",
"782":"JACK SMITH DRAW",
"783":"JACKSON LAKE",
"784":"JAMES CANYON",
"785":"JARALOSA CREEK",
"786":"JARITAS CREEK /NM",
"787":"JAROSA CREEK /N",
"788":"JAROSITO CANYON",
"789":"JAROSITO CREEK /N",
"790":"JEMEZ RIVER",
"791":"JERNIGAN WASH",
"792":"JIM GOODLOE CANYON /NM",
"793":"JOE CABIN ARROYO",
"794":"JOHNSON BASIN",
"795":"JOHNSON CANYON",
"796":"JOHNSON CREEK",
"797":"JONES ARROYO",
"798":"JONES CANYON /NM",
"799":"JORDAN CANYON",
"800":"JORNADA DEL MUERTO",
"801":"JORNADA DRAW",
"802":"JOSE PLANCENCIA CANYON",
"803":"JUAN TABO CANYON",
"804":"JUAN TAFOYA CANYON /N",
"805":"JUAN'S LAKE",
"806":"JUAQUILLA CREEK /N",
"807":"JUG CANYON",
"808":"JULIAN CANYON",
"809":"JUNIPER CANYON",
"810":"JUNIPER DRAW",
"811":"KANSAS VALLEY LAKE",
"812":"KAPPIS ARROYO",
"813":"KARR CANYON",
"814":"KARTCHNER ARROYO",
"815":"KEITHLY ARROYO /NM",
"816":"KELLER CANYON",
"817":"KELLY CANYON",
"818":"KELLY CANYON",
"819":"KIFFEN CANYON",
"820":"KIM-ME-NI-OLI WASH",
"821":"KIRKENDALL DRAW",
"822":"KOCHIS ARROYO",
"823":"KUTZ CANYON",
"824":"KYLE HARRISON CANYON",
"825":"L C CANYON",
"826":"LA BAJADA SEEP",
"827":"LA CANADA DE LA LOMA DE ARENA",
"828":"LA CANADA SANTIAGA",
"829":"LA CINTA",
"830":"LA DOUX ARROYO",
"831":"LA DRAW",
"832":"LA JARA CANYON",
"833":"LA JARA CANYON",
"834":"LA JARA CANYON",
"835":"LA JARA CANYON",
"836":"LA JARA CANYON",
"837":"LA JARA CREEK",
"838":"LA JARA CREEK",
"839":"LA JENCIA CREEK",
"840":"LA LUZ CANYON",
"841":"LA LUZ CREEK",
"842":"LA MANGA",
"843":"LA MANGA CREEK",
"844":"LA MARIPOSA FLOOD CONTROL",
"845":"LA MOSCA CANYON",
"846":"LA PALOMA CANYON",
"847":"LA PARITA CANYON",
"848":"LA PLATA RIVER",
"849":"LA POLVADERA CANYON",
"850":"LA QUESTA DE TRUJILLO",
"851":"LABORCITA ARROYO",
"852":"LABORCITA CANYON",
"853":"LADD ARROYO",
"854":"LADERA STORM DRAIN",
"855":"LAGUNA CANONEROS",
"856":"LAGUNA DEL PERRO",
"857":"LAKE ALICE",
"858":"LAKE AVALON",
"859":"LAKE CANYON",
"860":"LAKE FORK CANYON",
"861":"LAKE FORK RIO HONDO",
"862":"LAKE MALLOY",
"863":"LAKE MCMILLAN",
"864":"LAKE VALLEY DRAW /N",
"865":"LAKE VAN",
"866":"LAMPBRIGHT DRAW",
"867":"LAMY CANYON",
"868":"LARGA CANYON",
"869":"LARGO CANYON",
"870":"LARGO CANYON /N",
"871":"LARGO CREEK",
"872":"LAS ANIMAS CREEK",
"873":"LAS HUERTAS CREEK",
"874":"LAS TABLAS CANYON",
"875":"LAS YEGUAS CANYON",
"876":"LAST CHANCE CANYON",
"877":"LAST CHANCE DRAW",
"878":"LATIR LAKE CREEK",
"879":"LEA PLATEAU AREA 1",
"880":"LEA PLATEAU AREA 2",
"881":"LEA PLATEAU AREA 3",
"882":"LEA PLATEAU AREA 4",
"883":"LEACHMAN DRAW",
"884":"LEAD MINE CANYON",
"885":"LEASBURG ARROYO",
"886":"LEFEBRES CREEK",
"887":"LEFT FORK SACATON DRAW",
"888":"LEGGETT CANYON",
"889":"LEWIS CANYON",
"890":"LINCOLN CANYON",
"891":"LINCOLN CANYON",
"892":"LITTLE BRUSHY CANYON",
"893":"LITTLE CHERRY CREEK",
"894":"LITTLE COYOTE CREEK",
"895":"LITTLE CREEK",
"896":"LITTLE DOG CANYON",
"897":"LITTLE DRY CANYON",
"898":"LITTLE DRY CREEK",
"899":"LITTLE FLORIDA DRAWS /N",
"900":"LITTLE HASPAROS CANYON",
"901":"LITTLE MCKITTRICK DRAW",
"902":"LITTLE PALLUCHE CANYON",
"903":"LITTLE PAT CANYON",
"904":"LITTLE RECHUELOS",
"905":"LITTLE WALNUT CREEK",
"906":"LOBO CANYON",
"907":"LOBO CANYON",
"908":"LOBO CREEK",
"909":"LOBO CREEK",
"910":"LOCKLER CANYON",
"911":"LONG ARROYO",
"912":"LONG CANYON",
"913":"LONG CANYON",
"914":"LONG CANYON",
"915":"LOOKOUT CANYON",
"916":"LORDSBURG DRAW",
"917":"LOS ALAMOS CANYON",
"918":"LOS CASTILLOS ARROYO /NM",
"919":"LOS CEDROS ARROYO /N",
"920":"LOS HUERROS CREEK",
"921":"LOS INDIOS CANYON",
"922":"LOS OJITOS SALADA",
"923":"LOS PINOS RIVER",
"924":"LOS TANOS CREEK",
"925":"LOST GRAVE CANYON",
"926":"LOST LAKE",
"927":"LOST RIVER",
"928":"LOWER DEER CREEK",
"929":"LUCAS CANYON",
"930":"LUMBRE CANYON",
"931":"LUNA CREEK",
"932":"LURANCE CANYON",
"933":"LUTZ CANYON",
"934":"LYNCH DRAW",
"935":"LYONS DRAW",
"936":"MACHO CANYON",
"937":"MACHO CANYON",
"938":"MACHO CREEK",
"939":"MADERA CANON",
"940":"MAESTAS CREEK",
"941":"MAGADO CREEK",
"942":"MALDONADO CANYON",
"943":"MALONE DRAW",
"944":"MANGAS CREEK",
"945":"MANGAS CREEK",
"946":"MANSFIELD WASH",
"947":"MANUELITAS CREEK",
"948":"MARGUERETTE CREEK /N",
"949":"MARIANO LAKE",
"950":"MARTHA CREEK",
"951":"MARTIN DRAW",
"952":"MARTINEZ CANYON",
"953":"MARTINEZ CANYON",
"954":"MASON DRAW",
"955":"MCDERMOTT ARROYO",
"956":"MCGAFFEY LAKE",
"957":"MCKITTRICK",
"958":"MCLANE DRAW",
"959":"MCLEOD ARROYO",
"960":"MEDIO CANYON",
"961":"MEDIO DEL DIA CANYON",
"962":"MEERSCHAUR CANYON",
"963":"MENTZEL CANYON",
"964":"MESA WELL CANYON",
"965":"MESCAL CANYON",
"966":"MESITA CREEK",
"967":"MESQUITE STREET STORM DRAIN",
"968":"MESTENO DRAW",
"969":"MIDDLE ARROYO",
"970":"MIDDLE CANYON",
"971":"MIDDLE DOG CANYON",
"972":"MIDDLE FORK BURRO CREEK /NM",
"973":"MIDDLE FORK FIVEMILE DRAW",
"974":"MIDDLE FORK GILA RIVER",
"975":"MIDDLE FORK LAKE",
"976":"MIDDLE FORK MASON DRAW /N",
"977":"MIDDLE FORK RED RIVER",
"978":"MIDDLE FORK SAPILLO CREEK",
"979":"MIDDLE PERCHA CREEK",
"980":"MIDDLE PONIL CREEK",
"981":"MILAGRO CANYON /N",
"982":"MILK LAKE",
"983":"MILK RANCH CANYON",
"984":"MILL CANYON",
"985":"MILLER CANYON /N",
"986":"MILLER CREEK",
"987":"MILLIGAN GULCH",
"988":"MILLS CANYON",
"989":"MIMBRES BASIN",
"990":"MIMBRES RIVER",
"991":"MINER CANYON",
"992":"MINERAL CREEK",
"993":"MIRANDA ARROYO",
"994":"MITT AND BAR CANYON",
"995":"MITTEN BAR CANYON /N",
"996":"MOCCASIN DRAW",
"997":"MOGOLLON CREEK",
"998":"MOGOTITO",
"999":"MONICA CANYON",
"1000":"MONTOYA ARROYO",
"1001":"MONTOYA CANYON",
"1002":"MONUMENT CANYON",
"1003":"MONUMENT DRAW",
"1004":"MOORE CANYON",
"1005":"MORA RIVER",
"1006":"MORAS CREEK",
"1007":"MORENO CREEK",
"1008":"MORPHY LAKE /N",
"1009":"MOSLEY CANYON",
"1010":"MOSLEY CANYON",
"1011":"MOSSMAN ARROYO",
"1012":"MUD SPRING CANYON",
"1013":"MUD SPRINGS CANYON",
"1014":"MUDHOLE DRAW",
"1015":"MULE CANYON",
"1016":"MULE CREEK",
"1017":"MULE SPRINGS CREEK",
"1018":"MUNIZ CANYON",
"1019":"MYERS CANYON",
"1020":"MYNDUS ARROYO /N",
"1021":"McALLISTER LAKE",
"1022":"McKNIGHT CANYON",
"1023":"NACIMIENTO CREEK",
"1024":"NANCE CANYON",
"1025":"NARANJO CANYON",
"1026":"NAVAJO RESERVOIR",
"1027":"NAVAJO RIVER",
"1028":"NEGRITO CREEK",
"1029":"NEGRO CANYON",
"1030":"NEGRO CANYON",
"1031":"NEGRO ED CANYON",
"1032":"NELSON CANYON",
"1033":"NESTER CANYON",
"1034":"NESTER DRAW",
"1035":"NEW TANK DRAW",
"1036":"NEW TANK DRAW",
"1037":"NEWMAN CANYON",
"1038":"NIELSEN WATERSHED",
"1039":"NINEMILE CREEK",
"1040":"NINETYSIX CREEK",
"1041":"NOGAL ARROYO /N",
"1042":"NOGAL CANYON /N",
"1043":"NOGAL CANYON /N",
"1044":"NOGAL CANYON /N",
"1045":"NOGAL CREEK /N",
"1046":"NOGAL DRAW",
"1047":"NORTH BRANCH CORRUMPA CREEK",
"1048":"NORTH CANADIAN RIVER",
"1049":"NORTH CANYON",
"1050":"NORTH COPPER CANYON",
"1051":"NORTH COTTONWOOD CREEK",
"1052":"NORTH DRY CREEK",
"1053":"NORTH FORK ALAMOCITA CANYON",
"1054":"NORTH FORK CLARK CANYON",
"1055":"NORTH FORK CORRAL CANYON",
"1056":"NORTH FORK DEVILS CREEK",
"1057":"NORTH FORK GREEN CANYON",
"1058":"NORTH FORK LITTLE COYOTE CREEK",
"1059":"NORTH FORK NEGRO ED CANYON",
"1060":"NORTH FORK PALOMAS CREEK",
"1061":"NORTH FORK RIO LA CASA",
"1062":"NORTH FORK RIO QUEMADO",
"1063":"NORTH FORK SYCAMORE CREEK",
"1064":"NORTH FORK URRACA CREEK",
"1065":"NORTH FORK WALNUT CANYON",
"1066":"NORTH HOLLOW CREEK",
"1067":"NORTH MILL ARROYO",
"1068":"NORTH PERCHA CREEK",
"1069":"NORTH PLAINS",
"1070":"NORTH PONIL CREEK",
"1071":"NORTH SALEM ARROYO",
"1072":"NORTH SEVEN RIVERS",
"1073":"NORTH SEVEN RIVERS",
"1074":"NORTH SPRING CANYON",
"1075":"NORTH TEXAS HILL CANYON /NM",
"1076":"NORTHRUP CANYON",
"1077":"NUTRIA NO. 1",
"1078":"NUTRIA NO. 2",
"1079":"OAK CANYON",
"1080":"OAK DRAW",
"1081":"OAK SPRING CREEK",
"1082":"OCATE CREEK",
"1083":"OJITO SECO",
"1084":"OJITOS CANYON",
"1085":"OJO CALIENTE",
"1086":"OJO DE GALLO SPRING",
"1087":"OJO DE LOS POSOS",
"1088":"OJO DE PALO BLANCO",
"1089":"OJO NEGRO CREEK",
"1090":"OJO REDONDO CANYON",
"1091":"OJO SARCO CREEK",
"1092":"OLD CANYON",
"1093":"OLD MAID CANYON",
"1094":"ORTEGA CANYON",
"1095":"OSO CREEK",
"1096":"OSO CREEK",
"1097":"OTTO DRAW",
"1098":"OUTLAW CANYON",
"1099":"OWL CANYON",
"1100":"OWL DRAW",
"1101":"OX CANYON",
"1102":"OX SPRING CANYON",
"1103":"PAIGE DRAW",
"1104":"PAJARITO ARROYO",
"1105":"PAJARITO CANYON",
"1106":"PAJARITO CANYON",
"1107":"PAJARITO CREEK",
"1108":"PAJARO CANYON",
"1109":"PALEO CREEK",
"1110":"PALIZA CANYON",
"1111":"PALLUCHE CANYON",
"1112":"PALLUCHE WASH",
"1113":"PALMILLA DRAW",
"1114":"PALO BLANCO",
"1115":"PALOMA CANYON",
"1116":"PALOMAS CREEK",
"1117":"PALOMAS GAP CREEK",
"1118":"PANCHUELA CREEK",
"1119":"PATOS CREEK",
"1120":"PATTERSON CANYON",
"1121":"PECOS RIVER AREA 1",
"1122":"PECOS RIVER AREA 2",
"1123":"PECOS RIVER AREA 3",
"1124":"PECOS RIVER AREA 4",
"1125":"PECOS RIVER AREA 5",
"1126":"PEDERNAL",
"1127":"PENA BLANCA ARROYO",
"1128":"PENASCO CANYON",
"1129":"PENNSYLVANIA CANYON",
"1130":"PEPPER CANYON",
"1131":"PEPPIN CANYON /N",
"1132":"PEPPIN CANYON /N",
"1133":"PERALTA CANYON",
"1134":"PERCHA CREEK",
"1135":"PERICO CREEK",
"1136":"PERK CANYON",
"1137":"PHILADELPHIA CANYON",
"1138":"PIEDRA LISA ARROYO",
"1139":"PIEDRA LUMBRE",
"1140":"PIEDRAS MARCADAS",
"1141":"PIERCE CANYON",
"1142":"PIGPEN CREEK",
"1143":"PINABETE CREEK",
"1144":"PINABETES CREEK",
"1145":"PINE CANYON",
"1146":"PINE CANYON",
"1147":"PINE CANYON",
"1148":"PINE CANYON",
"1149":"PINE CIENEGA CREEK",
"1150":"PINE CREEK",
"1151":"PINE TREE CANYON",
"1152":"PINKEY WRIGHT CANYON",
"1153":"PINO CANYON",
"1154":"PINON CREEK",
"1155":"PINON WASH",
"1156":"PINTADA ARROYO",
"1157":"PIONEER CREEK",
"1158":"PIPE LINE DRAW",
"1159":"PLACER CREEK",
"1160":"PLACITAS ARROYO",
"1161":"PLACITAS ARROYO",
"1162":"PLACITAS CREEK /N",
"1163":"PLAYAS BASIN",
"1164":"PLAYAS LAKE",
"1165":"PLAZA LARGA CREEK",
"1166":"POINT OF ROCKS CANYON",
"1167":"POJOAQUE RIVER",
"1168":"POLE CANYON",
"1169":"POLEO CREEK",
"1170":"POLVADERA CREEK",
"1171":"PONIL CREEK",
"1172":"POPE CANYON",
"1173":"PORTER DRAW",
"1174":"PORTER-WISENHUNT ARROYO",
"1175":"PORVENIR CANYON",
"1176":"POST OFFICE CANYON",
"1177":"POTATO CANYON",
"1178":"POVERTY CREEK",
"1179":"PRAIRIE CANYON",
"1180":"PRIDE DRAW",
"1181":"PRIEST CANYON",
"1182":"PROP CANYON",
"1183":"PUEBLO CREEK",
"1184":"PUERCO RIVER",
"1185":"PUERTO CREEK",
"1186":"PUMP CANYON",
"1187":"PURGATOIRE RIVER",
"1188":"QUEMADO CANYON",
"1189":"QUEMADO LAKE",
"1190":"QUERECHO PLAINS",
"1191":"RAFAEL CREEK /NM",
"1192":"RAILROAD CANYON",
"1193":"RAILROAD WASH",
"1194":"RAINBOW WASH",
"1195":"RALPH ARROYO",
"1196":"RANA CANYON",
"1197":"RANCHERIA CANYON",
"1198":"RATON ARROYO /N",
"1199":"RATON CREEK",
"1200":"RATTLESNAKE CANYON",
"1201":"RATTLESNAKE CREEK",
"1202":"RAYADO CREEK",
"1203":"RAYMAC ARROYO",
"1204":"RED BLUFF DRAW",
"1205":"RED CANYON",
"1206":"RED CANYON",
"1207":"RED LAKE",
"1208":"RED RIVER",
"1209":"RED RIVER",
"1210":"RED SPRING DRAW",
"1211":"REDROCK CANYON /N",
"1212":"REDROCK CANYON /N",
"1213":"REECE CANYON",
"1214":"REED THURMAN ARROYO",
"1215":"REUNION DRAW",
"1216":"REVENTON DRAW",
"1217":"REVUELTO CREEK",
"1218":"RHODES ARROYO",
"1219":"RICHARDSON CANYON",
"1220":"RICKETSON DRAW",
"1221":"RILEY CANYON",
"1222":"RINCON ARROYO",
"1223":"RINCON LARGO",
"1224":"RINCONADA CANYON /N",
"1225":"RINCONADA CREEK /N",
"1226":"RIO AGUA NEGRA /N",
"1227":"RIO BONITO",
"1228":"RIO BRAZOS",
"1229":"RIO CAPULIN /N",
"1230":"RIO CEBOLLA",
"1231":"RIO CEBOLLA",
"1232":"RIO CHAMA",
"1233":"RIO CHAMITA",
"1234":"RIO CHIQUITO /N",
"1235":"RIO CHIQUITO /N",
"1236":"RIO CHUPADERO",
"1237":"RIO DE ARENAS",
"1238":"RIO DE LAS TRAMPAS",
"1239":"RIO DE LAS VACAS",
"1240":"RIO DE LOS PINOS",
"1241":"RIO DE TRUCHAS",
"1242":"RIO DEL MEDIO",
"1243":"RIO DEL OSO",
"1244":"RIO DEL PLANO",
"1245":"RIO EN MEDIO",
"1246":"RIO EN MEDIO",
"1247":"RIO FELIX",
"1248":"RIO FERNANDO DE TAOS",
"1249":"RIO FRIJOLES",
"1250":"RIO GALLINA",
"1251":"RIO GRANDE AREA 0 (COLORADO)",
"1252":"RIO GRANDE AREA 1",
"1253":"RIO GRANDE AREA 2",
"1254":"RIO GRANDE AREA 3",
"1255":"RIO GRANDE AREA 4",
"1256":"RIO GRANDE AREA 5",
"1257":"RIO GRANDE AREA 6",
"1258":"RIO GRANDE DE RANCHO",
"1259":"RIO GUADALUPE",
"1260":"RIO HONDO",
"1261":"RIO HONDO",
"1262":"RIO LA CASA",
"1263":"RIO LUCERO",
"1264":"RIO MEDIO",
"1265":"RIO MOQUINO",
"1266":"RIO MORA",
"1267":"RIO NAMBE",
"1268":"RIO NUTRIA",
"1269":"RIO NUTRIAS",
"1270":"RIO NUTRITAS",
"1271":"RIO PAGUATE",
"1272":"RIO PENASCO",
"1273":"RIO PESCADO",
"1274":"RIO PUEBLO",
"1275":"RIO PUEBLO DE TAOS",
"1276":"RIO PUERCO",
"1277":"RIO PUERCO DE CHAMA",
"1278":"RIO QUEMADO",
"1279":"RIO RUIDOSO",
"1280":"RIO SALADO",
"1281":"RIO SALADO",
"1282":"RIO SAN ANTONIO",
"1283":"RIO SAN JOSE /N",
"1284":"RIO SAN LEONARDO",
"1285":"RIO SANTA BARBARA",
"1286":"RIO TESUQUE",
"1287":"RIO TIERRA AMARILLA",
"1288":"RIO TULAROSA",
"1289":"RIO TUSAS",
"1290":"RIO VALDEZ",
"1291":"RIO VALLECITOS /N",
"1292":"RIO YAQUI",
"1293":"RITO AZUL",
"1294":"RITO CEBOLLA",
"1295":"RITO CIENEGUILLA /N",
"1296":"RITO CIENEGUILLA /N",
"1297":"RITO CREEK",
"1298":"RITO CREEK",
"1299":"RITO DE AGUA NEGRA CHIQUITA /N",
"1300":"RITO DE GASCON",
"1301":"RITO DE LA LAMA",
"1302":"RITO DE LA OLLA",
"1303":"RITO DE LAS PALOMAS",
"1304":"RITO DE LAS SILLAS",
"1305":"RITO DE LOS FRIJOLES",
"1306":"RITO DE LOS PINOS",
"1307":"RITO DE TIERRA AMARILLA",
"1308":"RITO DEL BUEY /N",
"1309":"RITO DEL MEDIO",
"1310":"RITO DEL MEDIO",
"1311":"RITO DEL OJO",
"1312":"RITO DEL TANQUE",
"1313":"RITO ENCINO",
"1314":"RITO GARCIA",
"1315":"RITO GRIEGO /N",
"1316":"RITO JAROSO",
"1317":"RITO LA PRESA",
"1318":"RITO LECHE",
"1319":"RITO MORPHY",
"1320":"RITO PENAS NEGRAS",
"1321":"RITO PRIMERO",
"1322":"RITO QUEMAZON",
"1323":"RITO ROMERO /N",
"1324":"RITO SAN JOSE",
"1325":"RITO SECO",
"1326":"ROAD CANYON",
"1327":"ROAD CREEK",
"1328":"ROBINSON DRAW",
"1329":"ROCK CANYON",
"1330":"ROCK CREEK",
"1331":"ROCK CREEK",
"1332":"ROCK HOUSE CANYON",
"1333":"ROCK HOUSE CANYON",
"1334":"ROCK HOUSE CANYON",
"1335":"ROCK LAKE",
"1336":"ROCK LAKE CREEK",
"1337":"ROCK SPRINGS CANYON",
"1338":"ROCK TANK CANYON",
"1339":"ROCK WATERHOLE CANYON",
"1340":"ROCKY ARROYO",
"1341":"RODEY ARROYO",
"1342":"ROMERO CANYON",
"1343":"ROUGH CANYON",
"1344":"ROUGH CREEK",
"1345":"RUSSIA CANYON",
"1346":"S CURVE CANYON",
"1347":"S U CANYON",
"1348":"SABINATA FLAT ARROYO",
"1349":"SACATON CREEK",
"1350":"SACATON DRAW",
"1351":"SACRAMENTO RIVER",
"1352":"SAGEBRUSH VALLEY",
"1353":"SALADITO CREEK",
"1354":"SALADO",
"1355":"SALADO ARROYO /NM",
"1356":"SALADO CANYON",
"1357":"SALADO CREEK",
"1358":"SALADO CREEK",
"1359":"SALADO CREEK",
"1360":"SALADO CREEK",
"1361":"SALADO CREEK",
"1362":"SALADO CREEK",
"1363":"SALADON CREEK",
"1364":"SALITRAL CREEK",
"1365":"SALIZ CANYON",
"1366":"SALOPEK ARROYO",
"1367":"SALT BASIN",
"1368":"SALT CREEK",
"1369":"SALT CREEK",
"1370":"SALT CREEK /N",
"1371":"SALT DRAW",
"1372":"SALTPETER CREEK",
"1373":"SAMBRITO CREEK",
"1374":"SAN ANDREAS CANYON",
"1375":"SAN ANTONIO CREEK",
"1376":"SAN ANTONIO CREEK /NM",
"1377":"SAN AUGUSTIN PLAINS",
"1378":"SAN CRISTOBAL",
"1379":"SAN FRANCISCO RIVER",
"1380":"SAN IGNACIO CREEK",
"1381":"SAN ISIDRO ARROYO /N",
"1382":"SAN ISIDRO CREEK",
"1383":"SAN ISIDRO WASH /N",
"1384":"SAN JON CREEK",
"1385":"SAN JOSE CANYON /N",
"1386":"SAN JUAN AREA 1",
"1387":"SAN JUAN AREA 2",
"1388":"SAN JUAN CANYON",
"1389":"SAN JUAN CANYON",
"1390":"SAN LEONARDO CANYON",
"1391":"SAN LORENZO ARROYO",
"1392":"SAN LUCAS CANYON",
"1393":"SAN MARCOS ARROYO",
"1394":"SAN MATEO CANYON",
"1395":"SAN MATEO CREEK",
"1396":"SAN MIGUEL CANYON /N",
"1397":"SAN MIGUEL CREEK /N",
"1398":"SAN PABLO CANYON",
"1399":"SAN PEDRO CREEK",
"1400":"SAN SIMON CIENEGA",
"1401":"SAN SIMON CREEK",
"1402":"SAN VICENTE ARROYO",
"1403":"SANCHEZ CANYON",
"1404":"SANCHEZ CANYON",
"1405":"SAND DRAW /N",
"1406":"SAND FLAT CANYON",
"1407":"SAND HILL ARROYO",
"1408":"SAND WASH",
"1409":"SANDY CANYON",
"1410":"SANGUIJUELA CREEK /N",
"1411":"SANTA CLARA CREEK",
"1412":"SANTA CRUZ RIVER",
"1413":"SANTA FE RIVER",
"1414":"SANTA RITA CANYON",
"1415":"SANTA RITA CREEK",
"1416":"SANTA TERESA",
"1417":"SANTIAGO CREEK /N",
"1418":"SAPELLO RIVER",
"1419":"SAPILLO CREEK",
"1420":"SARDINAS CANYON",
"1421":"SARGENT CANYON",
"1422":"SARGENT CANYON",
"1423":"SAUBLE ARROYO",
"1424":"SAUZ CREEK",
"1425":"SAVENNETA ARROYO /N",
"1426":"SAWMILL CANYON",
"1427":"SAWMILL CANYON",
"1428":"SAWMILL CREEK",
"1429":"SAWYER CREEK",
"1430":"SCHOOL SECTION CANYON",
"1431":"SEBOYETA CREEK",
"1432":"SEBOYETITA CREEK",
"1433":"SECO CANYON",
"1434":"SECO CREEK",
"1435":"SEEP SPRING DRAW",
"1436":"SEGREST DRAW",
"1437":"SENECA CREEK",
"1438":"SENORITO CANYON",
"1439":"SEVEN LAKES ARROYO",
"1440":"SEVENTEEN CANYON",
"1441":"SEVENTYSIX DRAW",
"1442":"SHAKESPEARE ARROYO",
"1443":"SHAW CANYON",
"1444":"SHEEP CAMP",
"1445":"SHEEP CANYON",
"1446":"SHEEP CORRAL DRAW",
"1447":"SHEEP PEN CANYON",
"1448":"SHELBY CLARK CANYON",
"1449":"SHELLY DITCH",
"1450":"SHIELDS CANYON",
"1451":"SHILOH DRAW",
"1452":"SHINGLE CANYON",
"1453":"SHIPMAN CANYON",
"1454":"SHOEMAKER CANYON",
"1455":"SHUMWAY ARROYO",
"1456":"SIBLEY CANON",
"1457":"SIBLEY GAP",
"1458":"SIEGREST DRAW /N",
"1459":"SILVA CANYON",
"1460":"SILVER CANYON",
"1461":"SILVER CITY DRAW /N",
"1462":"SILVER CREEK",
"1463":"SILVER SPRINGS CANYON",
"1464":"SILVER SPRINGS CREEK",
"1465":"SIMON CANYON",
"1466":"SIMPSON DRAW",
"1467":"SITTING BULL CANYON",
"1468":"SIXMILE CANYON",
"1469":"SIXMILE CREEK",
"1470":"SIXTEEN CANYON",
"1471":"SKULL CANYON",
"1472":"SKUNK CANYON",
"1473":"SKUTE STONE ARROYO",
"1474":"SLATE CREEK",
"1475":"SLOUGH CANYON",
"1476":"SMITH CANYON",
"1477":"SMUGGLER CREEK",
"1478":"SNAKY CANYON",
"1479":"SNARE CANYON",
"1480":"SNARE CANYON",
"1481":"SNOW CANYON",
"1482":"SODA POCKET CREEK",
"1483":"SOL SE METE CANYON",
"1484":"SOUTH BERRENDO CREEK",
"1485":"SOUTH BRANCH CORRUMPA CREEK",
"1486":"SOUTH COPPER CANYON",
"1487":"SOUTH COTTONWOOD CREEK",
"1488":"SOUTH FORK ARROYO LEON",
"1489":"SOUTH FORK BIG CREEK",
"1490":"SOUTH FORK CLARK CANYON",
"1491":"SOUTH FORK CORRAL CANYON",
"1492":"SOUTH FORK PALOMAS CREEK",
"1493":"SOUTH FORK PUERCO RIVER",
"1494":"SOUTH FORK ROAD CANYON",
"1495":"SOUTH FORK SILVER CREEK",
"1496":"SOUTH FORK SYCAMORE CREEK",
"1497":"SOUTH FORK URRACA CREEK",
"1498":"SOUTH FORK WALNUT CANYON",
"1499":"SOUTH FORK WHITEWATER CREEK",
"1500":"SOUTH PERCHA CREEK",
"1501":"SOUTH PONIL CREEK",
"1502":"SOUTH SEVEN RIVERS",
"1503":"SOUTH SPRING RIVER",
"1504":"SOUTH TANK CANYON",
"1505":"SOUTH WYLIE DRAW",
"1506":"SOUTHERN PACIFIC RESERVOIR",
"1507":"SPAR CANYON",
"1508":"SPRING ARROYO",
"1509":"SPRING ARROYO /NM",
"1510":"SPRING CANYON",
"1511":"SPRING CANYON",
"1512":"SPRING CANYON",
"1513":"SPRING CANYON",
"1514":"SPRING CREEK",
"1515":"SPRING CREEK",
"1516":"SPRING LAKE",
"1517":"SPRINGER ARROYO",
"1518":"SPUR TRAIL CANYON",
"1519":"STARKWEATHER CANYON",
"1520":"STARVATION DRAW",
"1521":"STEEP HOLLOW",
"1522":"STEEPLE ROCK CANYON",
"1523":"STEINS CREEK",
"1524":"STEVENS DRAW",
"1525":"STINKING DRAW",
"1526":"STINKING SPRING DRAW /N",
"1527":"STONE CANYON",
"1528":"STORRIE LAKE",
"1529":"STREET CANYON",
"1530":"STUBBLEFIELD ARROYO",
"1531":"STYCHINE DRAW /NM",
"1532":"SUGARITE CREEK",
"1533":"SURVEYORS CANYON",
"1534":"SWAN CANYON",
"1535":"SWEETWATER CREEK",
"1536":"SYCAMORE CREEK",
"1537":"TAFOYA CREEK /NM",
"1538":"TAIBAN CREEK",
"1539":"TALLEY CANYON",
"1540":"TAMPICO DRAW",
"1541":"TANBARK CANYON",
"1542":"TAPICITA CREEK",
"1543":"TATA VIQUE RIVER",
"1544":"TAYFOYA CANYON",
"1545":"TAYLOR CANYON",
"1546":"TAYLOR CREEK",
"1547":"TECOLOTE CEEK",
"1548":"TELEPHONE CANYON",
"1549":"TELEPHONE CANYON",
"1550":"TELEPHONE CANYON",
"1551":"TENNESSEE CREEK",
"1552":"TEQUESQUITE CREEK",
"1553":"TERRY CANYON",
"1554":"THIRTEEN MILE DRAW",
"1555":"THOMPSON CANYON",
"1556":"THOMPSON CANYON",
"1557":"THOMPSON DRAW",
"1558":"THREE RIVERS",
"1559":"THREEMILE CANYON",
"1560":"THURMAN DRAW",
"1561":"TIENDITAS CREEK",
"1562":"TIERRA BLANCA CREEK",
"1563":"TIFFANY CANYON",
"1564":"TIGRE ARROYO",
"1565":"TIJERAS CANYON",
"1566":"TIMBER CANYON",
"1567":"TINAJA CREEK",
"1568":"TOGEYE FLATS",
"1569":"TOM MOORE CANYON",
"1570":"TOMERLIN DRAW",
"1571":"TORREON DRAW",
"1572":"TORREON WASH",
"1573":"TORTOLITA CANYON",
"1574":"TORTOLITA CREEK",
"1575":"TORTUGAS ARROYO",
"1576":"TRABAJO CREEK",
"1577":"TRAMPEROS CREEK",
"1578":"TRAMWAY FLOODWATER",
"1579":"TRAVESSER CREEK",
"1580":"TRAVESSER CREEK",
"1581":"TRES HERMANOS",
"1582":"TRES LAGUNAS",
"1583":"TRIB OF TRIBUTARY",
"1584":"TRIB. OF MAJOR RIVER",
"1585":"TRINCHERA CREEK",
"1586":"TRINCHERA CREEK",
"1587":"TRINCHERITA CREEK /NM",
"1588":"TROUT CREEK",
"1589":"TROUT SPRINGS CANYON",
"1590":"TRUCHAS CREEK",
"1591":"TRUJILLO CANYON",
"1592":"TRUJILLO CREEK",
"1593":"TRUJILLO CREEK /N",
"1594":"TRUSDALE CANYON",
"1595":"TSE BONITA WASH",
"1596":"TUCKER DRAW",
"1597":"TUCUMCARI LAKE",
"1598":"TULAROSA BASIN",
"1599":"TULAROSA RIVER",
"1600":"TUMBLEWEED DRAW",
"1601":"TURKEY ARROYO /NM",
"1602":"TURKEY CANYON",
"1603":"TURKEY CANYON",
"1604":"TURKEY CREEK",
"1605":"TUSCOCOILLO CANYON",
"1606":"TWIN SISTERS CREEK",
"1607":"UHL DRAW",
"1608":"UNA DE GATO CREEK",
"1609":"UPPER CHARETTE LAKE",
"1610":"UPPER DOG CANYON",
"1611":"UPPER RANCH CANYON",
"1612":"UPPER RANCH CANYON",
"1613":"URRACA",
"1614":"URRACA CREEK",
"1615":"UTE CREEK",
"1616":"UTE CREEK",
"1617":"UTE RESERVOIR",
"1618":"UVAS VALLEY",
"1619":"VACINTE ARROYO /N",
"1620":"VALLE VIDAL",
"1621":"VALLECITO CREEK",
"1622":"VALLECITOS CREEK /N",
"1623":"VAN BREMMER CANYON",
"1624":"VANDERITAS CREEK /N",
"1625":"VAQUEROS CANYON",
"1626":"VEGALOSO CREEK",
"1627":"VELARDE ARROYO",
"1628":"VENTERO CREEK /N",
"1629":"VERMEJO CREEK",
"1630":"VERMEJO RIVER",
"1631":"VERNADO CANYON",
"1632":"VIGIL CANYON",
"1633":"VIGIL CANYON",
"1634":"VIGIL CANYON",
"1635":"VILLA MORA ARROYO",
"1636":"VINEYARD ARROYO",
"1637":"VOGHT DRAW",
"1638":"WALKER CANYON",
"1639":"WALNUT CANYON",
"1640":"WALNUT CREEK",
"1641":"WALNUT CREEK",
"1642":"WALNUT CREEK",
"1643":"WALNUT CREEK",
"1644":"WALNUT CREEK",
"1645":"WALNUT DRAW /N",
"1646":"WALNUT DRAW /N",
"1647":"WAMEL BASIN",
"1648":"WAMELS DRAW",
"1649":"WAMPOO WASH",
"1650":"WARDY HEDGECOCK ARROYO",
"1651":"WARM SPRINGS CANYON",
"1652":"WATER CANYON",
"1653":"WATER CANYON",
"1654":"WATER CANYON",
"1655":"WATER CANYON",
"1656":"WATER UNCON TO MAJOR",
"1657":"WATER UNCON TO MINOR",
"1658":"WATROUS CREEK",
"1659":"WAYLAND CANYON",
"1660":"WEBB CREEK",
"1661":"WELTY CANYON",
"1662":"WEST ANTELOPE DRAW",
"1663":"WEST DOG CANYON",
"1664":"WEST FORK GILA RIVER",
"1665":"WEST FORK TINAJA CREEK",
"1666":"WEST LATIR CREEK",
"1667":"WEST PRONG ASH CREEK",
"1668":"WEST RED CANYON",
"1669":"WEST SHIPPING PEN TANK",
"1670":"WESTWATER ARROYO",
"1671":"WHEATEN CREEK",
"1672":"WHISKEY CREEK",
"1673":"WHITE DEER CANYON",
"1674":"WHITE HORSE DRAW",
"1675":"WHITE LAKE",
"1676":"WHITE OAKS CANYON",
"1677":"WHITE ROCK CANYON",
"1678":"WHITEROCK CANYON",
"1679":"WHITES DRAW",
"1680":"WHITEWATER ARROYO",
"1681":"WHITEWATER CANYON",
"1682":"WHITEWATER CANYON",
"1683":"WHITEWATER CREEK",
"1684":"WHITEWATER CREEK",
"1685":"WHITEWATER CREEK",
"1686":"WHITMIRE CREEK",
"1687":"WILD HORSE CANYON",
"1688":"WILD HORSE CANYON /N",
"1689":"WILDCAN CANYON",
"1690":"WILDHORSE CANYON",
"1691":"WILLIE WHITE CANYON",
"1692":"WILLOW CANYON",
"1693":"WILLOW CREEK",
"1694":"WILLOW CREEK",
"1695":"WILLOW CREEK /N",
"1696":"WILLOW CREEK /N",
"1697":"WILLOW DRAW",
"1698":"WILLOW DRAW",
"1699":"WILLOW LAKE",
"1700":"WILLOW SPRING DRAW",
"1701":"WILLS CANYON",
"1702":"WILSON CANYON",
"1703":"WILSON CREEK",
"1704":"WIND CANYON",
"1705":"WIND MOUNTAIN DRAW",
"1706":"WINDHAM CANYON",
"1707":"WINN CANYON",
"1708":"WOLF CREEK",
"1709":"WOOD CANYON",
"1710":"WOOD CANYON",
"1711":"WOOD CANYON",
"1712":"WOODROW ARROYO /N",
"1713":"WOOTEN CANYON",
"1714":"WRIGHT CANYON /NM",
"1715":"WYLIE DRAW",
"1716":"WYNN CREEK",
"1717":"YANKEE CREEK /N",
"1718":"YELLOW POINT VALLEY",
"1719":"YESO CREEK",
"1720":"YL CANYON",
"1721":"YOAST DRAW",
"1722":"YORK CANYON",
"1723":"YOUNG CANYON",
"1724":"Z SLASH CANYON",
"1725":"ZEUFELDT ARROYO",
"1726":"ZUBER /N",
"1727":"ZUBER HOLLOW /N",
"1728":"ZUBI DRAW",
"1729":"ZUNI CANYON",
"1730":"ZUNI RIVER",
"1731":"BEAR SPRINGS CANYON",
"1732":"CHAVEZ CANYON",
"1733":"BEAR CANYON"}


def assignWaterSourceName(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = str(colrowValue)
        String1 = String1.strip()  # remove whitespace chars
        try:
            outList = waterSourceDict[String1]
        except:
            outList = ''
    return outList

dfinPOD['in_WaterSourceName'] = dfinPOD.apply(lambda row: assignWaterSourceName(row['surface_co']), axis=1)
dfinPOD['in_WaterSourceName'].unique()

array(['', 'DRY CIMARRON RIVER', 'RIO GRANDE AREA 5', 'RIO CHAMA',
       'RIO RUIDOSO', 'CHAVEZ CREEK', 'SAN FRANCISCO RIVER',
       'SAN JUAN AREA 1', 'DEEP CANYON', 'WILSON CANYON', 'AGUA CALIENTE',
       'RIO DEL OSO', 'LONG CANYON', 'BLACK RIVER', 'RIO TULAROSA',
       'DUCK CREEK', 'ADAMS CANYON', 'RIO PUEBLO DE TAOS',
       'CABRESTO CREEK', 'PALOMAS CREEK', 'ABO ARROYO', 'SAN JUAN AREA 2',
       'TRINCHERITA CREEK /NM', 'SPRING ARROYO /NM', 'WHITE OAKS CANYON',
       'ABBOTT LAKE', 'PECOS RIVER AREA 4', 'CASS DRAW', 'DARK CANYON /N',
       'CANONES CREEK /N', 'ALAMOSA CREEK', 'ABIQUIU CREEK',
       'PECOS RIVER AREA 5', 'ELEPHANT BUTTE RESERVOIR',
       'CORRALITOS CREEK', 'RIO BONITO', 'AGUA CHIQUITA CREEK',
       'SANTA FE RIVER', 'SANTA CLARA CREEK', 'RIO BRAZOS',
       'RIO CHUPADERO', 'RIO GRANDE AREA 3', 'RIO GRANDE AREA 1',
       'RIO GRANDE AREA 2', 'RIO TESUQUE', 'LA JARA CREEK', 'RIO CHAMITA',
       'BRAZOS RIVER', 'WILLOW CREEK /N', 'RITO DEL TANQUE',
  

In [8]:
#Creating Owner

def assignownerName(colrowValue1, colrowValue2):
    if colrowValue1 == '' or pd.isnull(colrowValue1):
        outList1 = ''
    else:
        outList1 = colrowValue1.strip()  # remove whitespace chars
    if colrowValue2 == '' or pd.isnull(colrowValue2):
        outList2 = ''
    else:
        outList2 = colrowValue2.strip()  # remove whitespace chars

    if outList1 == '' and outList2 == '':
        outList = ''
    elif outList1 == '':
        outList = outList2
    elif outList2 == '':
        outList = outList1
    else:
        outList = " ".join(map(str, [colrowValue1, colrowValue2]))
    return outList

dfinPOD['in_AllocationOwner'] = dfinPOD.apply(lambda row: assignownerName(row['own_fname'], row['own_lname']), axis=1)


def cleanOwnerDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().strip()
    return Val

dfinPOD['in_AllocationOwner'] = dfinPOD.apply(lambda row: cleanOwnerDataFunc(row['in_AllocationOwner']), axis=1)
dfinPOD['in_AllocationOwner'].unique()

array(['Laurence R And Diana M Berger', 'Koddy D Watson',
       'Roberta Targhetta', ..., 'Chris G Rodriguez', 'Jan Martin',
       'Billy Hull'], dtype=object)

In [9]:
benUseDict = {
    "AGR":"Agriculture other than irrigation",
    "AUG":"Augmentation well",
    "BPW":"Brine production well",
    "CEM":"Cemetery",
    "CLS":"Closed file",
    "COM":"Commercial",
    "CON":"Construction",
    "CPS":"Cathodic protection well",
    "DAI":"Dairy operation",
    "DCN":"Domestic construction",
    "DEW":"Dewatering well",
    "DOL":"72-12-1 domestic and livestock watering",
    "DOM":"72-12-1 domestic one household",
    "EXP":"Exploration",
    "FCD":"Flood control",
    "FGP":"Fish and game propogation",
    "FPO":"Feed pen operation",
    "GEO":"Geothermal boreholes",
    "HWY":"Highway construction",
    "IND":"Industrial",
    "INJ":"Injection",
    "IRR":"Irrigation",
    "MDW":"Community type use",
    "MFG":"Manufacturing",
    "MIL":"Military - military installations",
    "MIN":"Mining or milling or oil",
    "MOB":"Mobile home parks",
    "MON":"Monitoring well",
    "MPP":"Meat packing plant",
    "MUL":"72-12-1 multiple domestic households",
    "MUN":"Municipal - city or county supplied water",
    "N07":"No pre-1907 water right exists on this land",
    "NON":"Non-profit organizational use",
    "NOT":"No use of right or POD",
    "NRT":"No right",
    "OBS":"Observation",
    "OFM":"Oil field maintenance",
    "OIL":"Oil production",
    "PDL":"Non 72-12-1 domestic and livestock watering",
    "PDM":"Non 72-12-1 domestic one household",
    "PLS":"Non 72-12-1 livestock watering",
    "PMH":"Non 72-12-1 multiple domestic households",
    "POL":"Pollution control well",
    "POU":"Poultry and egg operation",
    "PPP":"Petroleum processing plant",
    "PRO":"72-12-1 Prospecting or development of natural resource",
    "PUB":"72-12-1 Construction of public works",
    "REC":"Recreation",
    "SAN":"72-12-1 Sanitary in conjunction with a commercial use",
    "SCH":"School use",
    "SRO":"Secondary recovery of oil",
    "STK":"72-12-1 livestock watering",
    "STO":"Storage",
    "STR":"Strategic water reserve",
    "SUB":"Subdivision",
    "SWR":"Stacked water right",
    "TBD":"To be determined",
    "UTL":"Public utility"}


def assignBenUseCategory(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = benUseDict[String1]
        except:
            outList = ''
    return outList

dfinPOD['in_BeneficialUseCategory'] = dfinPOD.apply(lambda row: assignBenUseCategory(row['use_']), axis=1)
dfinPOD['in_BeneficialUseCategory'].unique()

array(['72-12-1 livestock watering', 'Irrigation',
       '72-12-1 domestic one household',
       '72-12-1 Sanitary in conjunction with a commercial use', '',
       '72-12-1 multiple domestic households',
       '72-12-1 Prospecting or development of natural resource',
       '72-12-1 domestic and livestock watering', 'Closed file',
       'Non 72-12-1 livestock watering', 'No use of right or POD',
       'Recreation', 'Secondary recovery of oil', 'Subdivision',
       'No right', 'Exploration',
       'Non 72-12-1 domestic and livestock watering', 'Industrial',
       'Non 72-12-1 multiple domestic households', 'School use',
       'Commercial', 'Non 72-12-1 domestic one household',
       '72-12-1 Construction of public works',
       'Municipal - city or county supplied water', 'Monitoring well',
       'Highway construction', 'Community type use',
       'Pollution control well', 'Dairy operation',
       'Agriculture other than irrigation', 'Oil production',
       'Construction

In [10]:
AllocationLegalStatusDictionaryNM = {
    "ADJ":"Adjudicated",
    "ADM":"Administrative",
    "APP":"Application",
    "APR":"Application Being Protested",
    "CAN":"Cancelled",
    "CLS":"Closed File",
    "DCL":"Declaration",
    "DED":"Dedicated",
    "DEN":"Denied",
    "EXP":"Expired",
    "HS ":"Hydrographic Survey",
    "LIC":"Licensed",
    "NOI":"Notice of Intention",
    "NOT":"Not implies that there is no status",
    "OMS":"Owner Management Status",
    "OOJ":"Offer of Judgment",
    "PBU":"Proof of Beneficial Use",
    "PMT":"Permit",
    "PRG":"Purged Conversion Record",
    "REN":"Renumbered",
    "RET":"Retired",
    "TRN":"Transferred",
    "WMS":"Water Master Status",
    "WTD":"Withdrawn"    
}


def assignLegalStatus(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = AllocationLegalStatusDictionaryNM[String1]
        except:
            outList = ''
    return outList

dfinPOD['in_AllocationLegalStatusCV'] = dfinPOD.apply(lambda row: assignLegalStatus(row['status']), axis=1)
dfinPOD['in_AllocationLegalStatusCV'].unique()

array(['Declaration', 'Permit', 'Adjudicated', 'Licensed', 'Closed File',
       'Expired', '', 'Owner Management Status', 'Application',
       'Offer of Judgment', 'Cancelled',
       'Not implies that there is no status', 'Notice of Intention',
       'Transferred', 'Denied', 'Withdrawn', 'Renumbered',
       'Purged Conversion Record', 'Dedicated', 'Administrative',
       'Water Master Status', 'Retired'], dtype=object)

In [11]:
# County
CountyDictionaryNM = {
"BE" : "Bernalillo",
"CA" : "Catron",
"CH" : "Chaves",
"CI" : "Cibola",
"CO" : "Colfax",
"CU" : "Curry",
"DA" : "Dona Ana",
"DB" : "De Baca",
"ED" : "Eddy",
"GR" : "Grant",
"GU" : "Guadalupe",
"HA" : "Harding",
"HI" : "Hidalgo",
"LA" : "Los Alamos",
"LE" : "Lea",
"LI" : "Lincoln",
"LU" : "Luna",
"MK" : "McKinley",
"MO" : "Mora",
"OT" : "Otero",
"QU" : "Quay",
"RA" : "Rio Arriba",
"RO" : "Roosevelt",
"SA" : "Sandoval",
"SF" : "Santa Fe",
"SI" : "Sierra",
"SJ" : "San Juan",
"SM" : "San Miguel",
"SO" : "Socorro",
"TA" : "Taos",
"TO" : "Torrance",
"UN" : "Union",
"VA" : "Valencia",
"XX" : ""}


def assignCounty(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = CountyDictionaryNM[String1]
        except:
            outList = ''
    return outList

dfinPOD['in_County'] = dfinPOD.apply(lambda row: assignCounty(row['county']), axis=1)
dfinPOD['in_County'].unique()

array(['McKinley', 'Roosevelt', 'Sandoval', 'Lincoln', 'Socorro',
       'Chaves', 'Bernalillo', 'Valencia', 'Sierra', 'Dona Ana', 'Colfax',
       'Lea', 'Torrance', 'Santa Fe', 'Rio Arriba', 'Eddy', 'San Miguel',
       'Curry', 'Taos', 'Otero', 'Luna', 'Grant', '', 'Catron', 'Mora',
       'Harding', 'Cibola', 'Quay', 'Guadalupe', 'San Juan', 'Hidalgo',
       'De Baca', 'Union', 'Los Alamos'], dtype=object)

In [12]:
# create output POD dataframe
dfPOD = pd.DataFrame()

# Data Assessment UUID
dfPOD['WaDEUUID'] = dfinPOD['WaDEUUID']

# Method Info
dfPOD['in_MethodUUID'] = "NMwr_M1"

# Variable Info
dfPOD['in_VariableSpecificUUID'] = "NMwr_V1"

# Organization Info
dfPOD['in_OrganizationUUID'] = "NMwr_O1"

# WaterSource Info
dfPOD['in_Geometry'] = ""
dfPOD['in_GNISFeatureNameCV'] = ""
dfPOD['in_WaterQualityIndicatorCV'] = ""
dfPOD['in_WaterSourceName'] = dfinPOD['in_WaterSourceName'].str.title()
dfPOD['in_WaterSourceNativeID'] = "" # create customID for temp solution
dfPOD['in_WaterSourceTypeCV'] = dfinPOD['in_WaterSourceTypeCV']

# Site Info
dfPOD['in_CoordinateAccuracy'] = "WaDE Unspecified"
dfPOD['in_CoordinateMethodCV'] = "WaDE Unspecified"
dfPOD['in_County'] = dfinPOD['in_County']
dfPOD['in_Geometry'] = ""
dfPOD['in_GNISCodeCV'] = ""
dfPOD['in_HUC12'] = ""
dfPOD['in_HUC8'] = ""
dfPOD['in_Latitude'] = dfinPOD['in_Latitude']
dfPOD['in_Longitude'] = dfinPOD['in_Longitude']
dfPOD['in_NHDNetworkStatusCV'] = ""
dfPOD['in_NHDProductCV'] = ""
dfPOD['in_PODorPOUSite'] = "POD"
dfPOD['in_SiteName'] = dfinPOD['ditch_name'].str.title()
dfPOD['in_SiteNativeID'] = dfinPOD['pod_file'].str.strip()
dfPOD['in_SitePoint'] = ""
dfPOD['in_SiteTypeCV'] = "WaDE Unspecified"
dfPOD['in_StateCV'] = "NM"
dfPOD['in_USGSSiteID'] = ""

# AllocationAmount Info
dfPOD['in_AllocationApplicationDate'] = ""
dfPOD['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
dfPOD['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
dfPOD['in_AllocationBasisCV'] = ""
dfPOD['in_AllocationChangeApplicationIndicator'] = ""
dfPOD['in_AllocationCommunityWaterSupplySystem'] = ""
dfPOD['in_AllocationCropDutyAmount'] = ""
dfPOD['in_AllocationExpirationDate'] = ""
dfPOD['in_AllocationFlow_CFS'] = ""
dfPOD['in_AllocationLegalStatusCV'] = dfinPOD['in_AllocationLegalStatusCV']
dfPOD['in_AllocationNativeID'] =  dfinPOD['db_file'].str.strip()
dfPOD['in_AllocationOwner'] = dfinPOD['in_AllocationOwner']
dfPOD['in_AllocationPriorityDate'] = ""
dfPOD['in_AllocationSDWISIdentifierCV'] = ""
dfPOD['in_AllocationTimeframeEnd'] = ""
dfPOD['in_AllocationTimeframeStart'] = ""
dfPOD['in_AllocationTypeCV'] = ""
dfPOD['in_AllocationVolume_AF'] = dfinPOD['total_div']
dfPOD['in_BeneficialUseCategory'] = dfinPOD['in_BeneficialUseCategory']
dfPOD['in_CommunityWaterSupplySystem'] = ""
dfPOD['in_CropTypeCV'] = ""
dfPOD['in_CustomerTypeCV'] = ""
dfPOD['in_DataPublicationDate'] = ""
dfPOD['in_DataPublicationDOI'] = ""
dfPOD['in_ExemptOfVolumeFlowPriority'] = 1
dfPOD['in_GeneratedPowerCapacityMW'] = ""
dfPOD['in_IrrigatedAcreage'] = ""
dfPOD['in_IrrigationMethodCV'] = ""
dfPOD['in_LegacyAllocationIDs'] = ""
dfPOD['in_OwnerClassificationCV'] = ""
dfPOD['in_PopulationServed'] = ""
dfPOD['in_PowerType'] = ""
dfPOD['in_PrimaryBeneficialUseCategory'] = ""
dfPOD['in_SDWISIdentifierCV'] = ""
dfPOD['in_WaterAllocationNativeURL'] = dfinPOD['nmwrrs_wrs']


dfPOD = dfPOD.drop_duplicates().reset_index(drop=True)

print(len(dfPOD))
dfPOD.head()

260149


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,nmwr0,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,McKinley,,,,35.37273,-108.54187,,,POD,,G-01904,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,G-01904,Laurence R And Diana M Berger,,,,,,1.00000,72-12-1 livestock watering,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
1,nmwr1,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Roosevelt,,,,33.87138,-103.12541,,,POD,,CL-00128-POD1,,WaDE Unspecified,NM,,,,,,,,,,,Permit,CL-00128,Koddy D Watson,,,,,,120.00000,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
2,nmwr10,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,Groundwater,WaDE Unspecified,WaDE Unspecified,Sandoval,,,,35.24506,-106.59313,,,POD,,RG-68193,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,RG-68193,Roberta Targhetta,,,,,,16.52500,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
3,nmwr100,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Lincoln,,,,33.67663,-105.69425,,,POD,Benado Spring,SD-03176,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,SD-03176,Dr Phillip Kelsey Jeffers,,,,,,0.00000,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
4,nmwr1000,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Socorro,,,,33.88047,-107.30613,,,POD,,RG-72343,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,RG-72343,Arch Wilson,,,,,,3.00000,72-12-1 domestic one household,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...


In [13]:
# Concatenate dataframes
frames = [dfPOD] # add dataframes here
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

260149


## WaDE Custom Elements (due to missing sate info)

In [14]:
# Remove Special Characters
def removeSpecialCharFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).strip()
    return Val

In [15]:
# # some of their records differ by special characters only
# # we are going to try and leave special characters in, for now

# outdf['in_AllocationNativeID'] = outdf.apply(lambda row: removeSpecialCharFunc(row['in_AllocationNativeID']), axis=1)
# outdf['in_AllocationNativeID'].unique()

In [16]:
# # some of their records differ by special characters only
# # we are going to try and leave special characters in, for now

# outdf['in_SiteNativeID'] = outdf.apply(lambda row: removeSpecialCharFunc(row['in_SiteNativeID']), axis=1)
# outdf['in_SiteNativeID'].unique()

In [17]:
outdf['in_SiteName'] = outdf.apply(lambda row: removeSpecialCharFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['', 'Benado Spring', '2', ..., 'Maxwell Ditch',
       'Estes  Egbert No 1 N Ditch', 'Milliken No 1 N Ditch Enlarge'],
      dtype=object)

In [18]:
# Ensure Empty String

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [19]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['', 'Dry Cimarron River', 'Rio Grande Area 5', 'Rio Chama',
       'Rio Ruidoso', 'Chavez Creek', 'San Francisco River',
       'San Juan Area 1', 'Deep Canyon', 'Wilson Canyon', 'Agua Caliente',
       'Rio Del Oso', 'Long Canyon', 'Black River', 'Rio Tularosa',
       'Duck Creek', 'Adams Canyon', 'Rio Pueblo De Taos',
       'Cabresto Creek', 'Palomas Creek', 'Abo Arroyo', 'San Juan Area 2',
       'Trincherita Creek /Nm', 'Spring Arroyo /Nm', 'White Oaks Canyon',
       'Abbott Lake', 'Pecos River Area 4', 'Cass Draw', 'Dark Canyon /N',
       'Canones Creek /N', 'Alamosa Creek', 'Abiquiu Creek',
       'Pecos River Area 5', 'Elephant Butte Reservoir',
       'Corralitos Creek', 'Rio Bonito', 'Agua Chiquita Creek',
       'Santa Fe River', 'Santa Clara Creek', 'Rio Brazos',
       'Rio Chupadero', 'Rio Grande Area 3', 'Rio Grande Area 1',
       'Rio Grande Area 2', 'Rio Tesuque', 'La Jara Creek', 'Rio Chamita',
       'Brazos River', 'Willow Creek /N', 'Rito Del Tanque',
  

In [20]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['', 'Groundwater', 'Surface Water', 'Surface and Groundwater'],
      dtype=object)

In [21]:
outdf['in_CoordinateAccuracy'] = outdf.apply(lambda row: ensureEmptyString(row['in_CoordinateAccuracy']), axis=1)
outdf['in_CoordinateAccuracy'].unique()

array(['WaDE Unspecified'], dtype=object)

In [22]:
outdf['in_CoordinateMethodCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_CoordinateMethodCV']), axis=1)
outdf['in_CoordinateMethodCV'].unique()

array(['WaDE Unspecified'], dtype=object)

In [23]:
outdf['in_County'] = outdf.apply(lambda row: ensureEmptyString(row['in_County']), axis=1)
outdf['in_County'].unique()

array(['McKinley', 'Roosevelt', 'Sandoval', 'Lincoln', 'Socorro',
       'Chaves', 'Bernalillo', 'Valencia', 'Sierra', 'Dona Ana', 'Colfax',
       'Lea', 'Torrance', 'Santa Fe', 'Rio Arriba', 'Eddy', 'San Miguel',
       'Curry', 'Taos', 'Otero', 'Luna', 'Grant', '', 'Catron', 'Mora',
       'Harding', 'Cibola', 'Quay', 'Guadalupe', 'San Juan', 'Hidalgo',
       'De Baca', 'Union', 'Los Alamos'], dtype=object)

In [24]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['WaDE Unspecified'], dtype=object)

In [25]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['', 'Benado Spring', '2', ..., 'Maxwell Ditch',
       'Estes  Egbert No 1 N Ditch', 'Milliken No 1 N Ditch Enlarge'],
      dtype=object)

In [26]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['Declaration', 'Permit', 'Adjudicated', 'Licensed', 'Closed File',
       'Expired', '', 'Owner Management Status', 'Application',
       'Offer of Judgment', 'Cancelled',
       'Not implies that there is no status', 'Notice of Intention',
       'Transferred', 'Denied', 'Withdrawn', 'Renumbered',
       'Purged Conversion Record', 'Dedicated', 'Administrative',
       'Water Master Status', 'Retired'], dtype=object)

In [27]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['Laurence R And Diana M Berger', 'Koddy D Watson',
       'Roberta Targhetta', ..., 'Chris G Rodriguez', 'Jan Martin',
       'Billy Hull'], dtype=object)

In [28]:
outdf['in_AllocationTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationTypeCV']), axis=1)
outdf['in_AllocationTypeCV'].unique()

array([''], dtype=object)

In [29]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['72-12-1 livestock watering', 'Irrigation',
       '72-12-1 domestic one household',
       '72-12-1 Sanitary in conjunction with a commercial use', '',
       '72-12-1 multiple domestic households',
       '72-12-1 Prospecting or development of natural resource',
       '72-12-1 domestic and livestock watering', 'Closed file',
       'Non 72-12-1 livestock watering', 'No use of right or POD',
       'Recreation', 'Secondary recovery of oil', 'Subdivision',
       'No right', 'Exploration',
       'Non 72-12-1 domestic and livestock watering', 'Industrial',
       'Non 72-12-1 multiple domestic households', 'School use',
       'Commercial', 'Non 72-12-1 domestic one household',
       '72-12-1 Construction of public works',
       'Municipal - city or county supplied water', 'Monitoring well',
       'Highway construction', 'Community type use',
       'Pollution control well', 'Dairy operation',
       'Agriculture other than irrigation', 'Oil production',
       'Construction

In [30]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

array([''], dtype=object)

In [31]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

array([1.0, 120.0, 16.525, ..., 34.238, 19.18, 45.6], dtype=object)

In [32]:
# Fixing in_IrrigatedAcreage datatype
outdf['in_IrrigatedAcreage'] = pd.to_numeric(outdf['in_IrrigatedAcreage'], errors='coerce').replace(0,"").fillna("")
outdf['in_IrrigatedAcreage'].unique()

array([''], dtype=object)

In [33]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna("")
outdf['in_Latitude'].unique()

array([35.37273145, 33.87137855, 35.24506086, ..., 34.64802436,
       35.23059876, 35.29889858])

In [34]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna("")
outdf['in_Longitude'].unique()

array([-108.54186809, -103.12540585, -106.59312684, ..., -103.2507427 ,
       -106.70562355, -106.45923071])

In [35]:
%%time

# Creating WaDE Custom water source native ID for easy water source identification
# Change 'outstring' name to be state specific.
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

Wall time: 1min 6s


array(['', 'wadeID2', 'wadeID3', 'wadeID4', 'wadeID5', 'wadeID6',
       'wadeID7', 'wadeID8', 'wadeID9', 'wadeID10', 'wadeID11',
       'wadeID12', 'wadeID13', 'wadeID14', 'wadeID15', 'wadeID16',
       'wadeID17', 'wadeID18', 'wadeID19', 'wadeID20', 'wadeID21',
       'wadeID22', 'wadeID23', 'wadeID24', 'wadeID25', 'wadeID26',
       'wadeID27', 'wadeID28', 'wadeID29', 'wadeID30', 'wadeID31',
       'wadeID32', 'wadeID33', 'wadeID34', 'wadeID35', 'wadeID36',
       'wadeID37', 'wadeID38', 'wadeID39', 'wadeID40', 'wadeID41',
       'wadeID42', 'wadeID43', 'wadeID44', 'wadeID45', 'wadeID46',
       'wadeID47', 'wadeID48', 'wadeID49', 'wadeID50', 'wadeID51',
       'wadeID52', 'wadeID53', 'wadeID54', 'wadeID55', 'wadeID56',
       'wadeID57', 'wadeID58', 'wadeID59', 'wadeID60', 'wadeID61',
       'wadeID62', 'wadeID63', 'wadeID64', 'wadeID65', 'wadeID66',
       'wadeID67', 'wadeID68', 'wadeID69', 'wadeID70', 'wadeID71',
       'wadeID72', 'wadeID73', 'wadeID74', 'wadeID75', 'wadeID76',

## Review and Export

In [36]:
outdf.dtypes

WaDEUUID                           object
in_MethodUUID                      object
in_VariableSpecificUUID            object
in_OrganizationUUID                object
in_Geometry                        object
                                    ...  
in_PopulationServed                object
in_PowerType                       object
in_PrimaryBeneficialUseCategory    object
in_SDWISIdentifierCV               object
in_WaterAllocationNativeURL        object
Length: 62, dtype: object

In [37]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,nmwr0,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,McKinley,,,,35.37273,-108.54187,,,POD,,G-01904,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,G-01904,Laurence R And Diana M Berger,,,,,,1.00000,72-12-1 livestock watering,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
1,nmwr1,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Roosevelt,,,,33.87138,-103.12541,,,POD,,CL-00128-POD1,,WaDE Unspecified,NM,,,,,,,,,,,Permit,CL-00128,Koddy D Watson,,,,,,120.00000,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
2,nmwr10,NMwr_M1,NMwr_V1,NMwr_O1,,,,,wadeID2,Groundwater,WaDE Unspecified,WaDE Unspecified,Sandoval,,,,35.24506,-106.59313,,,POD,,RG-68193,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,RG-68193,Roberta Targhetta,,,,,,16.52500,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
3,nmwr100,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Lincoln,,,,33.67663,-105.69425,,,POD,Benado Spring,SD-03176,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,SD-03176,Dr Phillip Kelsey Jeffers,,,,,,,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
4,nmwr1000,NMwr_M1,NMwr_V1,NMwr_O1,,,,,,,WaDE Unspecified,WaDE Unspecified,Socorro,,,,33.88047,-107.30613,,,POD,,RG-72343,,WaDE Unspecified,NM,,,,,,,,,,,Declaration,RG-72343,Arch Wilson,,,,,,3.00000,72-12-1 domestic one household,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260144,nmwr99995,NMwr_M1,NMwr_V1,NMwr_O1,,,,,wadeID2,Groundwater,WaDE Unspecified,WaDE Unspecified,Chaves,,,,33.23508,-104.37462,,,POD,,RA-09076-CLW,,WaDE Unspecified,NM,,,,,,,,,,,Permit,RA-09076,Raymond Jordan,,,,,,3.00000,72-12-1 multiple domestic households,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
260145,nmwr99996,NMwr_M1,NMwr_V1,NMwr_O1,,,,,wadeID2,Groundwater,WaDE Unspecified,WaDE Unspecified,Curry,,,,34.64802,-103.25074,,,POD,,CC-00893-S6,,WaDE Unspecified,NM,,,,,,,,,,,Permit,CC-00893,Nn Dairy A Nm Partnership,,,,,,1440.00000,Irrigation,,,,,,1,,,,,,,,,,http://nmwrrs.ose.state.nm.us/ReportDispatcher...
260146,nmwr99997,NMwr_M1,NMwr_V1,NMwr_O1,,,,,wadeID2,Groundwater,WaDE Unspecified,WaDE Unspecified,Sandoval,,,,35.23060,-106.70562,,,POD,,RG-63584-POD1,,WaDE Unspecified,NM,,,,,,,,,,,Permit,RG-63584,Jan Martin,,,,,,3.00000,72-12-1 multiple domestic households,,,,,,1,,,,,,,,,,http://nmw

In [38]:
# Export the output dataframe
outdf.to_csv('Pwr_nmMain.zip', index=False, compression="zip")  # The output, save as a zip